# Set up libraries

In [1]:
# Use the os package to interact with the environment
import os

# Bring in Pandas for Dataframe functionality
import pandas as pd

# numpy for basics
import numpy as np

# Use StringIO for working with file contents
from io import StringIO

# Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

# Import seaborn for plots
import seaborn as sns

# Enable interaction with the FireCloud API
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

# BigQuery for querying data
from google.cloud import bigquery

In [2]:
#!pip install rpy2
#Install rpy2 if not already installed - uncomment if running for the first time

In [3]:
import rpy2.rinterface

# Set up billing project and data path variables

In [4]:
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']

WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

GS_RELEASE_PATH = 'gs://amp-pd-data/releases/2021_v2-5release_0510'
GS_CLINICAL_RELEASE_PATH = f'{GS_RELEASE_PATH}/clinical'

GS_WGS_RELEASE_PATH = 'gs://amp-pd-genomics/releases/2021_v2-5release_0510/wgs'
GS_WGS_RELEASE_PLINK_PATH = os.path.join(GS_WGS_RELEASE_PATH, 'plink')
GS_WGS_RELEASE_GATK_PATH = os.path.join(GS_WGS_RELEASE_PATH, 'gatk')

BQ_RELEASE_DATASET = 'amp-pd-research.2021_v2_5release_0510'

print(BILLING_PROJECT_ID)
print(GS_CLINICAL_RELEASE_PATH)
print(GS_WGS_RELEASE_PLINK_PATH)
print(GS_WGS_RELEASE_GATK_PATH)

manuela-norway
gs://amp-pd-data/releases/2021_v2-5release_0510/clinical
gs://amp-pd-genomics/releases/2021_v2-5release_0510/wgs/plink
gs://amp-pd-genomics/releases/2021_v2-5release_0510/wgs/gatk


# Set up functions

In [5]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}')
    !$command

# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))
    
# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)
    
# Get the data from a query
def bq_query(query):
    """Return the contents of a query against BigQuery"""
    return pd.read_gbq(
        query,
        project_id=BILLING_PROJECT_ID,
        dialect='standard')

# Copy pathway lists to workspace

In [10]:
%%bash
ls /home/jupyter/genetic_data_v2-5

allPD_HapMap_geneticPCs.tab
AMP_forEuroPCA.bed
AMP_forEuroPCA.bim
AMP_forEuroPCA.fam
AMP_forEuroPCA.log
AMP_forEuroPCA.pruned.bed
AMP_forEuroPCA.pruned.bim
AMP_forEuroPCA.pruned.fam
AMP_forEuroPCA.pruned.log
AMP_forEuroPCA.pruning.log
AMP_forEuroPCA.pruning.prune.in
AMP_forEuroPCA.pruning.prune.out
AMP_HapMap_merged.filtered.bed
AMP_HapMap_merged.filtered.bim
AMP_HapMap_merged.filtered.fam
AMP_HapMap_merged.filtered.log
AMP_HapMap_merged.filtered.pruned.bed
AMP_HapMap_merged.filtered.pruned.bim
AMP_HapMap_merged.filtered.pruned.fam
AMP_HapMap_merged.filtered.pruned.log
AMP.QCed_chrbp.txt
AMP.QCed.updated_names.bed
AMP.QCed.updated_names.bim
AMP.QCed.updated_names.fam
AMP.QCed.updated_names.log
AMP.QCed.updated_names.no_dupli.bed
AMP.QCed.updated_names.no_dupli.bim
AMP.QCed.updated_names.no_dupli.fam
AMP.QCed.updated_names.no_dupli.log
AMP_toMergeHapMap.bed
AMP_toMergeHapMap.bim
AMP_toMergeHapMap.fam
AMP_toMergeHapMap.log
APOE.log
APOE.raw
BioFIND.pca.eigenval
BioFIND.pca.eigenvec
BioFI

Make new directory for pathway lists

In [10]:
%%bash
mkdir -p /home/jupyter/pathways

In [12]:
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp {WORKSPACE_BUCKET}/*_extract_hg38.txt /home/jupyter/pathways/')

Executing: gsutil -mu manuela-norway cp gs://fc-2f037edd-3b8a-43cc-8283-b6704707b181/*_extract_hg38.txt /home/jupyter/pathways/
Copying gs://fc-2f037edd-3b8a-43cc-8283-b6704707b181/adaptive_immune_extract_hg38.txt...
Copying gs://fc-2f037edd-3b8a-43cc-8283-b6704707b181/alpha_synuclein_extract_hg38.txt...
Copying gs://fc-2f037edd-3b8a-43cc-8283-b6704707b181/microglia_extract_hg38.txt...
Copying gs://fc-2f037edd-3b8a-43cc-8283-b6704707b181/endocytosis_extract_hg38.txt...
Copying gs://fc-2f037edd-3b8a-43cc-8283-b6704707b181/mitochondria_extract_hg38.txt...
Copying gs://fc-2f037edd-3b8a-43cc-8283-b6704707b181/monocytes_extract_hg38.txt...
Copying gs://fc-2f037edd-3b8a-43cc-8283-b6704707b181/innate_immune_extract_hg38.txt...
Copying gs://fc-2f037edd-3b8a-43cc-8283-b6704707b181/lysosome_extract_hg38.txt...
/ [8/8 files][  2.1 MiB/  2.1 MiB] 100% Done                                    
Operation completed over 8 objects/2.1 MiB.                                      


In [7]:
%%bash
ls /home/jupyter/pathways

adaptive_immune.bed
adaptive_immune.bim
adaptive_immune_chrbp.txt
adaptive_immune_extract_hg38.txt
adaptive_immune.fam
adaptive_immune.log
adaptive_immune_PD_PRS_Chang23andMe.all_score
adaptive_immune_PD_PRS_Chang23andMe.log
adaptive_immune_PD_PRS_Chang23andMe.mismatch
adaptive_immune_PD_PRS_Chang23andMe.prsice
adaptive_immune.updated_names.bed
adaptive_immune.updated_names.bim
adaptive_immune.updated_names.fam
adaptive_immune.updated_names.log
adaptive_immune.updated_names.no_dupli.bed
adaptive_immune.updated_names.no_dupli.bim
adaptive_immune.updated_names.no_dupli.fam
adaptive_immune.updated_names.no_dupli.log
alpha_synuclein.bed
alpha_synuclein.bim
alpha_synuclein_chrbp.txt
alpha_synuclein_extract_hg38.txt
alpha_synuclein.fam
alpha_synuclein.log
alpha_synuclein_PD_PRS_Chang23andMe.all_score
alpha_synuclein_PD_PRS_Chang23andMe.log
alpha_synuclein_PD_PRS_Chang23andMe.prsice
alpha_synuclein.updated_names.bed
alpha_synuclein.updated_names.bim
alpha_synuclein.updated_names.fam
alpha_syn

# For each pathway, extract regions from AMP-PD QC'ed genetic data

This is done on the full AMP-PD genetic data and not separated by cohorts yet. As the cohorts were joint called and the genetic data is in a merged file, the same variants are present across all cohorts. Later we will analyse the PRSs separately in each cohort.

In [13]:
%%bash
for PATHWAY in adaptive_immune alpha_synuclein innate_immune lysosome endocytosis mitochondria microglia monocytes
do
/home/jupyter/plink2 --bfile /home/jupyter/genetic_data_v2-5/merged.PDonly.updated_sex.mind_0.05.het.binary.xsplit.nongenetic.no_dupli.auto_snps.european.pihat.geno.missing_hap.hwe.maf \
            --extract range /home/jupyter/pathways/"$PATHWAY"_extract_hg38.txt \
            --exclude range /home/jupyter/genetic_data_v2-5/exclusion_regions_hg38.txt \
            --make-bed \
            --out /home/jupyter/pathways/"$PATHWAY" \
            --threads 16
done

PLINK v2.00a3LM AVX2 Intel (26 Aug 2021)       www.cog-genomics.org/plink/2.0/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/pathways/adaptive_immune.log.
Options in effect:
  --bfile /home/jupyter/genetic_data_v2-5/merged.PDonly.updated_sex.mind_0.05.het.binary.xsplit.nongenetic.no_dupli.auto_snps.european.pihat.geno.missing_hap.hwe.maf
  --exclude range /home/jupyter/genetic_data_v2-5/exclusion_regions_hg38.txt
  --extract range /home/jupyter/pathways/adaptive_immune_extract_hg38.txt
  --make-bed
  --out /home/jupyter/pathways/adaptive_immune
  --threads 16

Start time: Thu Jun  9 13:20:36 2022
60292 MiB RAM detected; reserving 30146 MiB for main workspace.
Using up to 16 threads (change this with --threads).
2610 samples (951 females, 1659 males; 2610 founders) loaded from
/home/jupyter/genetic_data_v2-5/merged.PDonly.updated_sex.mind_0.05.het.binary.xsplit.nongenetic.no_dupli.auto_snps.european.pihat.geno.missing_hap.hwe.maf

# Install PRSice2

In [14]:
%%bash

if test -e /home/jupyter/PRSice_linux; then

echo "PRSice2 is already installed in /home/jupyter/"
else
echo "PRSice2 is not installed"
cd /home/jupyter/

wget https://github.com/choishingwan/PRSice/releases/download/2.3.3/PRSice_linux.zip

unzip -o PRSice_linux.zip
fi

PRSice2 is already installed in /home/jupyter/


In [15]:
%%bash
ls /home/jupyter/

AMP-PD_Pathway_PRS
clinical_data_v2-5
entrypoint.out
gcta_1.93.2beta
gcta_1.93.2beta.zip
genetic_data_v2-5
HY3_results
jupyter.log
LDpred2
LICENSE
lost+found
notebooks
packages
pathways
plink
plink2
plink2_linux_avx2_20210826.zip
plink_linux_x86_64_20190304.zip
prettify
PRSice_linux
PRSice_linux.zip
PRSice.R
R_packages
SNCA
sumstats
TOY_BASE_GWAS.assoc
toy.map
toy.ped
TOY_TARGET_DATA.bed
TOY_TARGET_DATA.bim
TOY_TARGET_DATA.fam
TOY_TARGET_DATA.pheno
welder.log


# Copy Chang 2017 summary stats to workspace and tidy

Using Chang 2017 GWAS discovery to create PRSs: http://research-pub.gene.com/chang_et_al_2017/

INCLUDING 23andMe data (need contract with 23andMe)

In [16]:
%%bash
mkdir -p /home/jupyter/sumstats
cd /home/jupyter/sumstats
mkdir -p Chang2017_no23andMe

In [16]:
%%bash
cd /home/jupyter/sumstats
mkdir -p Chang2017_with23andMe
mkdir -p Chang2017_with23andMe_Sara

In [18]:
%%bash
ls /home/jupyter/sumstats

Chang2017_no23andMe
Chang2017_with23andMe


In [16]:
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp {WORKSPACE_BUCKET}/PD_all_xmeta.dat  /home/jupyter/sumstats/Chang2017_with23andMe')
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp {WORKSPACE_BUCKET}/all_snp_info.txt  /home/jupyter/sumstats/Chang2017_with23andMe')

Executing: gsutil -mu manuela-norway cp gs://fc-2f037edd-3b8a-43cc-8283-b6704707b181/PD_all_xmeta.dat  /home/jupyter/sumstats/Chang2017_with23andMe
Copying gs://fc-2f037edd-3b8a-43cc-8283-b6704707b181/PD_all_xmeta.dat...
\ [1/1 files][  1.5 GiB/  1.5 GiB] 100% Done  55.3 MiB/s ETA 00:00:00           
Operation completed over 1 objects/1.5 GiB.                                      
Executing: gsutil -mu manuela-norway cp gs://fc-2f037edd-3b8a-43cc-8283-b6704707b181/all_snp_info.txt  /home/jupyter/sumstats/Chang2017_with23andMe
Copying gs://fc-2f037edd-3b8a-43cc-8283-b6704707b181/all_snp_info.txt...
| [1/1 files][  7.1 GiB/  7.1 GiB] 100% Done  50.6 MiB/s ETA 00:00:00           
Operation completed over 1 objects/7.1 GiB.                                      


In [19]:
%%bash
ls /home/jupyter/sumstats/Chang2017_with23andMe

all_snp_info.txt
chang_37_for_liftOver.bed
chang_37_forPRSice.txt
Chang_hg38_forPRSice.tab
Chang_rsids.txt
liftOver_output.bed
PD_all_xmeta.dat


There are currently issues with the 23andMe sumstats that 23andMe sent, as they are truncated at chr4 and 23andMe have not replied to any emails. Sara has kindly sent over another version of the Chang2017 sumstats including 23andMe data. I have lifted this over to hg38 and formatted for PRSice.

In [8]:
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp {WORKSPACE_BUCKET}/META4_hg38.tab  /home/jupyter/sumstats/Chang2017_with23andMe_Sara')

Executing: gsutil -mu manuela-norway cp gs://fc-2f037edd-3b8a-43cc-8283-b6704707b181/META4_hg38.tab  /home/jupyter/sumstats/Chang2017_with23andMe_Sara
Copying gs://fc-2f037edd-3b8a-43cc-8283-b6704707b181/META4_hg38.tab...
/ [1/1 files][314.4 MiB/314.4 MiB] 100% Done                                    
Operation completed over 1 objects/314.4 MiB.                                    


# Format Chang2017 summary stats

The data is currently in two files
* PD_all_xmeta.dat has the GWAS summary stats information with a unique number identifier for each SNP
* all_snp_info.txt has the SNP information, chr, position, alleles etc. in GRCh37

This data is for European ancestry GWAS only.

Need to merge these files.

In [17]:
#First read in both files
Chang = pd.read_csv(f'/home/jupyter/sumstats/Chang2017_with23andMe/PD_all_xmeta.dat',
                   delimiter = "\t")

snps = pd.read_csv(f'/home/jupyter/sumstats/Chang2017_with23andMe/all_snp_info.txt',
                  delimiter = "\t")

In [18]:
#Merge files by all.data.id (unique identifier for each SNP), keeping only SNPs in PD_all_xmeta.dat
Chang_snps = pd.merge(Chang, snps, left_on = 'all.data.id', right_on = 'all.data.id', how = 'left')

In [19]:
Chang.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15635593 entries, 0 to 15635592
Data columns (total 16 columns):
 #   Column       Dtype  
---  ------       -----  
 0   all.data.id  int64  
 1   src          object 
 2   pvalue       float64
 3   effect       float64
 4   stderr       float64
 5   pass         object 
 6   im.num.0     float64
 7   dose.b.0     float64
 8   im.num.1     float64
 9   dose.b.1     float64
 10  AA.0         float64
 11  AB.0         float64
 12  BB.0         float64
 13  AA.1         float64
 14  AB.1         float64
 15  BB.1         float64
dtypes: float64(13), int64(1), object(2)
memory usage: 1.9+ GB


In [20]:
Chang_snps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15635593 entries, 0 to 15635592
Data columns (total 33 columns):
 #   Column        Dtype  
---  ------        -----  
 0   all.data.id   int64  
 1   src           object 
 2   pvalue        float64
 3   effect        float64
 4   stderr        float64
 5   pass          object 
 6   im.num.0      float64
 7   dose.b.0      float64
 8   im.num.1      float64
 9   dose.b.1      float64
 10  AA.0          float64
 11  AB.0          float64
 12  BB.0          float64
 13  AA.1          float64
 14  AB.1          float64
 15  BB.1          float64
 16  gt.data.id    float64
 17  im.data.id    float64
 18  assay.name    object 
 19  scaffold      object 
 20  position      int64  
 21  alleles       object 
 22  ploidy        object 
 23  cytoband      object 
 24  gene.context  object 
 25  is.v1         bool   
 26  is.v2         bool   
 27  is.v3         bool   
 28  is.v4         bool   
 29  is.v5         bool   
 30  h550          bo

In [21]:
#Remove SNPs that have not passed QC
Chang_snps_pass = Chang_snps.copy()
Chang_snps_pass = Chang_snps_pass[(Chang_snps_pass['pass']== "Y")]

In [22]:
Chang_snps_pass.head()

,all.data.id,src,pvalue,effect,stderr,pass,im.num.0,dose.b.0,im.num.1,dose.b.1,...,cytoband,gene.context,is.v1,is.v2,is.v3,is.v4,is.v5,h550,omni,strand
2,3,I,0.771412,-0.009939,0.034239,Y,159956.0,0.847624,4947.0,0.848227,...,1p36.33,[]--OR4F5,False,False,False,False,False,False,False,+
5,6,I,0.788298,0.007978,0.029746,Y,159956.0,0.177142,4947.0,0.176896,...,1p36.33,[]--OR4F5,False,False,False,False,False,False,False,+
7,8,I,0.917844,-0.002636,0.025539,Y,302034.0,0.823991,6476.0,0.823804,...,1p36.33,[]--OR4F5,False,False,False,False,False,False,False,+
9,10,I,0.787689,0.008319,0.030927,Y,302034.0,0.173884,6476.0,0.173288,...,1p36.33,[]--OR4F5,False,False,False,False,False,False,False,+
10,11,I,0.820159,0.007291,0.032136,Y,159956.0,0.161230,4947.0,0.160554,...,1p36.33,[]--OR4F5,False,False,False,False,False,False,False,+


In [23]:
#Select just relevant columns
Chang_snps_pass = Chang_snps_pass[['all.data.id', 'pvalue', 'effect', 'stderr', 'assay.name', 'scaffold', 'position', 'alleles']]

In [24]:
Chang_snps_pass.head()

,all.data.id,pvalue,effect,stderr,assay.name,scaffold,position,alleles
2,3,0.771412,-0.009939,0.034239,rs555500075,chr1,10352,D/I
5,6,0.788298,0.007978,0.029746,rs558604819,chr1,10642,A/G
7,8,0.917844,-0.002636,0.025539,rs544419019,chr1,11012,C/G
9,10,0.787689,0.008319,0.030927,rs574746232,chr1,13011,G/T
10,11,0.820159,0.007291,0.032136,rs540538026,chr1,13110,A/G


In [25]:
#Split alleles into two columns. Note that the order of the alleles is just alphabetical so will call alleleA and B.
#Note that B is the effect allele in the summary statistics
Chang_snps_pass_formatted = Chang_snps_pass.copy()
Chang_snps_pass_formatted = Chang_snps_pass_formatted.reset_index(drop = True)
Chang_snps_pass_formatted[['alleleA', 'alleleB']] = Chang_snps_pass_formatted['alleles'].str.split('/', expand = True)
Chang_snps_pass_formatted.head()

,all.data.id,pvalue,effect,stderr,assay.name,scaffold,position,alleles,alleleA,alleleB
0,3,0.771412,-0.009939,0.034239,rs555500075,chr1,10352,D/I,D,I
1,6,0.788298,0.007978,0.029746,rs558604819,chr1,10642,A/G,A,G
2,8,0.917844,-0.002636,0.025539,rs544419019,chr1,11012,C/G,C,G
3,10,0.787689,0.008319,0.030927,rs574746232,chr1,13011,G/T,G,T
4,11,0.820159,0.007291,0.032136,rs540538026,chr1,13110,A/G,A,G


In [26]:
#Make new column for chromosome without 'chr'
Chang_snps_pass_formatted['chr'] = Chang_snps_pass_formatted['scaffold'].str.replace(r'chr', '').astype(str)

#Make BP str format
Chang_snps_pass_formatted["position"] = Chang_snps_pass_formatted["position"].astype(str)

#Make combined column chrbp
Chang_snps_pass_formatted['chrbp'] = Chang_snps_pass_formatted['chr'] + ":" + Chang_snps_pass_formatted["position"]

In [21]:
Chang_snps_pass_formatted.head()

,all.data.id,pvalue,effect,stderr,assay.name,scaffold,position,alleles,alleleA,alleleB,chr,chrbp
0,3,0.771412,-0.009939,0.034239,rs555500075,chr1,10352,D/I,D,I,1,1:10352
1,6,0.788298,0.007978,0.029746,rs558604819,chr1,10642,A/G,A,G,1,1:10642
2,8,0.917844,-0.002636,0.025539,rs544419019,chr1,11012,C/G,C,G,1,1:11012
3,10,0.787689,0.008319,0.030927,rs574746232,chr1,13011,G/T,G,T,1,1:13011
4,11,0.820159,0.007291,0.032136,rs540538026,chr1,13110,A/G,A,G,1,1:13110


In [27]:
export_chang_hg37 = Chang_snps_pass_formatted.copy()
export_chang_hg37 = export_chang_hg37[['chrbp', 'alleleB', 'alleleA', 'effect', 'pvalue', 'stderr']]

#Rename effect column to BETA
export_chang_hg37 = export_chang_hg37.rename(columns={'effect': 'BETA'})

export_chang_hg37.head()

,chrbp,alleleB,alleleA,BETA,pvalue,stderr
0,1:10352,I,D,-0.009939,0.771412,0.034239
1,1:10642,G,A,0.007978,0.788298,0.029746
2,1:11012,G,C,-0.002636,0.917844,0.025539
3,1:13011,T,G,0.008319,0.787689,0.030927
4,1:13110,G,A,0.007291,0.820159,0.032136


In [28]:
#Export hg37 data
with open('/home/jupyter/sumstats/Chang2017_with23andMe/chang_37_forPRSice.txt', 'w') as f:
    f.write(export_chang_hg37.to_csv(index=False, header = True, sep='\t'))

In [29]:
#Copy to workspace
shell_do(f'gsutil -u {BILLING_PROJECT_ID} cp -r /home/jupyter/sumstats/Chang2017_with23andMe/chang_37_forPRSice.txt {WORKSPACE_BUCKET}')

Executing: gsutil -u manuela-norway cp -r /home/jupyter/sumstats/Chang2017_with23andMe/chang_37_forPRSice.txt gs://fc-2f037edd-3b8a-43cc-8283-b6704707b181
Copying file:///home/jupyter/sumstats/Chang2017_with23andMe/chang_37_forPRSice.txt [Content-Type=text/plain]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

- [1 files][550.8 MiB/550.8 MiB]   51.9 MiB/s  

# Liftover Chang2017 summary stats to hg38

I cannot get liftOver working on Terra because 'cannot execute binary file: Exec format error'. So I have run liftOver on my local machine and copied the liftOver output files into the workspace

I have checked liftover coordinates against BioMart coordinates and they match perfectly. There are some instances where BioMart is missing coords because the rsID has been merged with another, and one SNP which is not lifted over in liftOver.

Read in Chang 2017 summary stats and format for liftover

In [30]:
Chang_snps_pass_formatted.head()

,all.data.id,pvalue,effect,stderr,assay.name,scaffold,position,alleles,alleleA,alleleB,chr,chrbp
0,3,0.771412,-0.009939,0.034239,rs555500075,chr1,10352,D/I,D,I,1,1:10352
1,6,0.788298,0.007978,0.029746,rs558604819,chr1,10642,A/G,A,G,1,1:10642
2,8,0.917844,-0.002636,0.025539,rs544419019,chr1,11012,C/G,C,G,1,1:11012
3,10,0.787689,0.008319,0.030927,rs574746232,chr1,13011,G/T,G,T,1,1:13011
4,11,0.820159,0.007291,0.032136,rs540538026,chr1,13110,A/G,A,G,1,1:13110


Write just rsID list for bioMart to get positions in hg38

In [31]:
chang_rsids = Chang_snps_pass_formatted.copy()
chang_rsids = chang_rsids[['assay.name']]

In [32]:
with open('/home/jupyter/sumstats/Chang2017_with23andMe/Chang_rsids.txt', 'w') as f:
    f.write(chang_rsids.to_csv(index=False, header = False, sep='\t'))

In [33]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} cp -r /home/jupyter/sumstats/Chang2017_with23andMe/Chang_rsids.txt {WORKSPACE_BUCKET}')

Executing: gsutil -u manuela-norway cp -r /home/jupyter/sumstats/Chang2017_with23andMe/Chang_rsids.txt gs://fc-2f037edd-3b8a-43cc-8283-b6704707b181
Copying file:///home/jupyter/sumstats/Chang2017_with23andMe/Chang_rsids.txt [Content-Type=text/plain]...
/ [1 files][144.1 MiB/144.1 MiB]                                                
Operation completed over 1 objects/144.1 MiB.                                    


For liftOver you need chr$CHR, start and end bp

In [39]:
Chang_snps_pass_formatted.head()

,all.data.id,pvalue,effect,stderr,assay.name,scaffold,position,alleles,alleleA,alleleB,chr,chrbp
0,3,0.771412,-0.009939,0.034239,rs555500075,chr1,10352,D/I,D,I,1,1:10352
1,6,0.788298,0.007978,0.029746,rs558604819,chr1,10642,A/G,A,G,1,1:10642
2,8,0.917844,-0.002636,0.025539,rs544419019,chr1,11012,C/G,C,G,1,1:11012
3,10,0.787689,0.008319,0.030927,rs574746232,chr1,13011,G/T,G,T,1,1:13011
4,11,0.820159,0.007291,0.032136,rs540538026,chr1,13110,A/G,A,G,1,1:13110


Add 1 base to create BP_end. Must first convert position column to numeric

In [44]:
chang_for_liftover = Chang_snps_pass_formatted.copy()
chang_for_liftover['BP_end'] = pd.to_numeric(chang_for_liftover['position']) + 1

In [45]:
chang_for_liftover.head()

,all.data.id,pvalue,effect,stderr,assay.name,scaffold,position,alleles,alleleA,alleleB,chr,chrbp,BP_end
0,3,0.771412,-0.009939,0.034239,rs555500075,chr1,10352,D/I,D,I,1,1:10352,10353
1,6,0.788298,0.007978,0.029746,rs558604819,chr1,10642,A/G,A,G,1,1:10642,10643
2,8,0.917844,-0.002636,0.025539,rs544419019,chr1,11012,C/G,C,G,1,1:11012,11013
3,10,0.787689,0.008319,0.030927,rs574746232,chr1,13011,G/T,G,T,1,1:13011,13012
4,11,0.820159,0.007291,0.032136,rs540538026,chr1,13110,A/G,A,G,1,1:13110,13111


In [46]:
chang_for_liftover = chang_for_liftover[['scaffold', 'position', 'BP_end', 'assay.name']]

In [47]:
chang_for_liftover.head()

,scaffold,position,BP_end,assay.name
0,chr1,10352,10353,rs555500075
1,chr1,10642,10643,rs558604819
2,chr1,11012,11013,rs544419019
3,chr1,13011,13012,rs574746232
4,chr1,13110,13111,rs540538026


In [48]:
with open('/home/jupyter/sumstats/Chang2017_with23andMe/chang_37_for_liftOver.bed', 'w') as f:
    f.write(chang_for_liftover.to_csv(index=False, header = False, sep='\t'))

In [49]:
%%bash
tail /home/jupyter/sumstats/Chang2017_with23andMe/chang_37_for_liftOver.bed

chr4	44604298	44604299	rs59905107
chr4	44604490	44604491	rs370922792
chr4	44604516	44604517	rs193293102
chr4	44604548	44604549	rs62306189
chr4	44604601	44604602	rs4473688
chr4	44604765	44604766	rs535845793
chr4	44604776	44604777	rs566170966
chr4	44604782	44604783	rs539975122
chr4	44604876	44604877	rs556428381
chr4	44604897	44604898	rs190343291


Copy formatted file to workspace bucket

In [50]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} cp -r /home/jupyter/sumstats/Chang2017_with23andMe/chang_37_for_liftOver.bed {WORKSPACE_BUCKET}')

Executing: gsutil -u manuela-norway cp -r /home/jupyter/sumstats/Chang2017_with23andMe/chang_37_for_liftOver.bed gs://fc-2f037edd-3b8a-43cc-8283-b6704707b181
Copying file:///home/jupyter/sumstats/Chang2017_with23andMe/chang_37_for_liftOver.bed [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

- [1 files][437.4 MiB/437.4

Now run liftOver on local machine using command like below (or online)

In [42]:
#shell_do(f'/home/jupyter-user/notebooks/liftOver \
#        /home/jupyter-user/notebooks/sumstats/Chang2017_no23andMe/PD_meta_analysis_PDGENE_PDWBS_liftOver.tab \
#        hg19ToHg38.over.chain \
#        /home/jupyter-user/notebooks/sumstats/Chang2017_no23andMe/liftOver_output.bed \
#        /home/jupyter-user/notebooks/sumstats/Chang2017_no23andMe/liftOver_unlifted.bed')

#NOT RUN - LIFTOVER NOT WORKING ON TERRA

Executing: /home/jupyter-user/notebooks/liftOver         /home/jupyter-user/notebooks/sumstats/Chang2017_no23andMe/PD_meta_analysis_PDGENE_PDWBS_liftOver.tab         hg19ToHg38.over.chain         /home/jupyter-user/notebooks/sumstats/Chang2017_no23andMe/liftOver_output.bed         /home/jupyter-user/notebooks/sumstats/Chang2017_no23andMe/liftOver_unlifted.bed
/bin/sh: 1: /home/jupyter-user/notebooks/liftOver: Exec format error


Copy liftOver output into workspace bucket

In [51]:
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp {WORKSPACE_BUCKET}/liftOver_output.bed /home/jupyter/sumstats/Chang2017_with23andMe')

Executing: gsutil -mu manuela-norway cp gs://fc-2f037edd-3b8a-43cc-8283-b6704707b181/liftOver_output.bed /home/jupyter/sumstats/Chang2017_with23andMe
Copying gs://fc-2f037edd-3b8a-43cc-8283-b6704707b181/liftOver_output.bed...
| [1/1 files][437.4 MiB/437.4 MiB] 100% Done                                    
Operation completed over 1 objects/437.4 MiB.                                    


In [52]:
Chang_snps_pass_formatted.head()

,all.data.id,pvalue,effect,stderr,assay.name,scaffold,position,alleles,alleleA,alleleB,chr,chrbp
0,3,0.771412,-0.009939,0.034239,rs555500075,chr1,10352,D/I,D,I,1,1:10352
1,6,0.788298,0.007978,0.029746,rs558604819,chr1,10642,A/G,A,G,1,1:10642
2,8,0.917844,-0.002636,0.025539,rs544419019,chr1,11012,C/G,C,G,1,1:11012
3,10,0.787689,0.008319,0.030927,rs574746232,chr1,13011,G/T,G,T,1,1:13011
4,11,0.820159,0.007291,0.032136,rs540538026,chr1,13110,A/G,A,G,1,1:13110


Merge sumstats with liftover 

In [53]:
#Read in liftOver output
liftover = pd.read_csv(f'/home/jupyter/sumstats/Chang2017_with23andMe/liftOver_output.bed',
                   delimiter = "\t", header = None)

In [56]:
liftover.columns=['CHR', 'BP_start', 'BP_end', 'rsid']
liftover.head()

,CHR,BP_start,BP_end,rsid
0,chr1,10352,10353,rs555500075
1,chr1,10642,10643,rs558604819
2,chr1,11012,11013,rs544419019
3,chr1,13011,13012,rs574746232
4,chr1,13110,13111,rs540538026


IMPORTANT: The BP_start is the correct coordinates for the SNP in hg38

Merge sumstats with liftover output - keeping only the SNPs that have been successfully lifted over

In [57]:
merged = pd.merge(Chang_snps_pass_formatted, liftover, left_on='assay.name', right_on='rsid', how = 'left')
merged.head()

,all.data.id,pvalue,effect,stderr,assay.name,scaffold,position,alleles,alleleA,alleleB,chr,chrbp,CHR,BP_start,BP_end,rsid
0,3,0.771412,-0.009939,0.034239,rs555500075,chr1,10352,D/I,D,I,1,1:10352,chr1,10352.0,10353.0,rs555500075
1,6,0.788298,0.007978,0.029746,rs558604819,chr1,10642,A/G,A,G,1,1:10642,chr1,10642.0,10643.0,rs558604819
2,8,0.917844,-0.002636,0.025539,rs544419019,chr1,11012,C/G,C,G,1,1:11012,chr1,11012.0,11013.0,rs544419019
3,10,0.787689,0.008319,0.030927,rs574746232,chr1,13011,G/T,G,T,1,1:13011,chr1,13011.0,13012.0,rs574746232
4,11,0.820159,0.007291,0.032136,rs540538026,chr1,13110,A/G,A,G,1,1:13110,chr1,13110.0,13111.0,rs540538026


In [58]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12896220 entries, 0 to 12896219
Data columns (total 16 columns):
 #   Column       Dtype  
---  ------       -----  
 0   all.data.id  int64  
 1   pvalue       float64
 2   effect       float64
 3   stderr       float64
 4   assay.name   object 
 5   scaffold     object 
 6   position     object 
 7   alleles      object 
 8   alleleA      object 
 9   alleleB      object 
 10  chr          object 
 11  chrbp        object 
 12  CHR          object 
 13  BP_start     float64
 14  BP_end       float64
 15  rsid         object 
dtypes: float64(5), int64(1), object(10)
memory usage: 1.6+ GB


In [59]:
#Check SNPs that were not lifted over
merged[pd.isnull(merged['BP_start'])]

,all.data.id,pvalue,effect,stderr,assay.name,scaffold,position,alleles,alleleA,alleleB,chr,chrbp,CHR,BP_start,BP_end,rsid
25085,29157,0.342646,-0.098071,0.102126,rs560097800,chr1,1584097,C/T,C,T,1,1:1584097,NaN,NaN,NaN,NaN
25086,29158,0.631880,0.020462,0.042813,rs28525262,chr1,1584102,A/G,A,G,1,1:1584102,NaN,NaN,NaN,NaN
25087,29160,0.390288,0.022722,0.026506,rs4648756,chr1,1584117,C/G,C,G,1,1:1584117,NaN,NaN,NaN,NaN
25090,29163,0.266015,0.028806,0.025969,rs4648607,chr1,1584218,C/T,C,T,1,1:1584218,NaN,NaN,NaN,NaN
25091,29164,0.178164,0.033290,0.024796,rs4648757,chr1,1584224,C/G,C,G,1,1:1584224,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12198266,14781142,0.921972,-0.002974,0.030403,rs34433551,chr4,9556102,C/T,C,T,4,4:9556102,NaN,NaN,NaN,NaN
12198267,14781143,0.948783,0.002074,0.032316,rs192467273,chr4,9556105,G/T,G,T,4,4:9556105,NaN,NaN,NaN,NaN
12198520,14781438,0.980380,-0.003491,0.142193,rs34899372,chr4,9561627,C/T,C,T,4,4:9561627,NaN,NaN,NaN,NaN
12198539,14781458,0.493654,-0.180768,0.269673,rs2061994,chr4,9561927,C/T,C,T,4,4:9561927,NaN,NaN,NaN,NaN


In [61]:
#Remove SNPs that were not lifted over (and any rows where there are any NAs)
merged_nomiss = merged.copy()
merged_nomiss = merged_nomiss.dropna()

In [62]:
merged_nomiss.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12894461 entries, 0 to 12896219
Data columns (total 16 columns):
 #   Column       Dtype  
---  ------       -----  
 0   all.data.id  int64  
 1   pvalue       float64
 2   effect       float64
 3   stderr       float64
 4   assay.name   object 
 5   scaffold     object 
 6   position     object 
 7   alleles      object 
 8   alleleA      object 
 9   alleleB      object 
 10  chr          object 
 11  chrbp        object 
 12  CHR          object 
 13  BP_start     float64
 14  BP_end       float64
 15  rsid         object 
dtypes: float64(5), int64(1), object(10)
memory usage: 1.6+ GB


In [63]:
merged_nomiss.head()

,all.data.id,pvalue,effect,stderr,assay.name,scaffold,position,alleles,alleleA,alleleB,chr,chrbp,CHR,BP_start,BP_end,rsid
0,3,0.771412,-0.009939,0.034239,rs555500075,chr1,10352,D/I,D,I,1,1:10352,chr1,10352.0,10353.0,rs555500075
1,6,0.788298,0.007978,0.029746,rs558604819,chr1,10642,A/G,A,G,1,1:10642,chr1,10642.0,10643.0,rs558604819
2,8,0.917844,-0.002636,0.025539,rs544419019,chr1,11012,C/G,C,G,1,1:11012,chr1,11012.0,11013.0,rs544419019
3,10,0.787689,0.008319,0.030927,rs574746232,chr1,13011,G/T,G,T,1,1:13011,chr1,13011.0,13012.0,rs574746232
4,11,0.820159,0.007291,0.032136,rs540538026,chr1,13110,A/G,A,G,1,1:13110,chr1,13110.0,13111.0,rs540538026


# Format Chang SNP names to harmonise between sumstats and target data

In the Chang2017 sumstats, make a new column for the SNP name which is chr:bp - to match AMP data
(We are not naming SNPs with as chr:bp:A2:A1 because the order of the alleles is not the same in the Chang data)

In [64]:
merged_formatted = merged_nomiss.copy()

#First make a column for chr which is just the chromosome number. Convert to string format
merged_formatted['chr'] = merged_formatted['scaffold'].str.replace(r'chr', '').astype(str)

#Round BP_start to remove decimal places
merged_formatted["BP_start"] = merged_formatted["BP_start"].round().astype(int)

#Make BP str format
merged_formatted["BP_start"] = merged_formatted["BP_start"].astype(str)

#Make combined column chrbp
merged_formatted['chrbp'] = merged_formatted['chr'] + ":" + merged_formatted["BP_start"]

In [65]:
merged_formatted.tail()

,all.data.id,pvalue,effect,stderr,assay.name,scaffold,position,alleles,alleleA,alleleB,chr,chrbp,CHR,BP_start,BP_end,rsid
12896215,15630329,0.341476,-0.019517,0.020535,rs535845793,chr4,44604765,A/G,A,G,4,4:44602748,chr4,44602748,44602749.0,rs535845793
12896216,15630330,0.419701,-0.021318,0.026467,rs566170966,chr4,44604776,C/T,C,T,4,4:44602759,chr4,44602759,44602760.0,rs566170966
12896217,15630331,0.964053,0.003946,0.087707,rs539975122,chr4,44604782,C/T,C,T,4,4:44602765,chr4,44602765,44602766.0,rs539975122
12896218,15630335,0.785134,-0.007339,0.026939,rs556428381,chr4,44604876,A/G,A,G,4,4:44602859,chr4,44602859,44602860.0,rs556428381
12896219,15630336,0.329613,0.019403,0.019921,rs190343291,chr4,44604897,C/T,C,T,4,4:44602880,chr4,44602880,44602881.0,rs190343291


In [66]:
merged_formatted_export = merged_formatted.copy()
merged_formatted_export = merged_formatted_export[['chrbp','chr', 'BP_start', 'alleleB', 'alleleA', 'pvalue', 'effect']]
merged_formatted_export.head()

,chrbp,chr,BP_start,alleleB,alleleA,pvalue,effect
0,1:10352,1,10352,I,D,0.771412,-0.009939
1,1:10642,1,10642,G,A,0.788298,0.007978
2,1:11012,1,11012,G,C,0.917844,-0.002636
3,1:13011,1,13011,T,G,0.787689,0.008319
4,1:13110,1,13110,G,A,0.820159,0.007291


In [67]:
#Rename columns

#Note that in the Chang summary statistics (read the documentation provided by 23andMe, the effect statistic is for the B allele, the alphabetically higher allele)
#So alleleB is going to be A1 - in PRSice A1 is the effect allele
merged_formatted_export.columns = ['chrbp', 'CHR', 'BP', 'A1', 'A2', 'P', 'beta']
merged_formatted_export.head()

,chrbp,CHR,BP,A1,A2,P,beta
0,1:10352,1,10352,I,D,0.771412,-0.009939
1,1:10642,1,10642,G,A,0.788298,0.007978
2,1:11012,1,11012,G,C,0.917844,-0.002636
3,1:13011,1,13011,T,G,0.787689,0.008319
4,1:13110,1,13110,G,A,0.820159,0.007291


In [114]:
merged_formatted_export.shape

(12894461, 7)

In [123]:
#Remove duplicates by chrbp
merged_formatted_export_nodupli = merged_formatted_export.copy()

#First remove SNPs with D/I alleles
merged_formatted_export_nodupli = merged_formatted_export_nodupli[(merged_formatted_export_nodupli['A1'] !="I") & (merged_formatted_export_nodupli['A1']!="D")]

#Now remove duplicate SNPs by chrbp
merged_formatted_export_nodupli = merged_formatted_export_nodupli.drop_duplicates(subset = ['chrbp'])
merged_formatted_export_nodupli.shape

(12022304, 7)

In [124]:
with open('/home/jupyter/sumstats/Chang2017_with23andMe/Chang_hg38_forPRSice.tab', 'w') as f:
    f.write(merged_formatted_export_nodupli.to_csv(index=False, sep='\t'))

In [20]:
%%bash
head /home/jupyter/sumstats/Chang2017_with23andMe/Chang_hg38_forPRSice.tab

chrbp	CHR	BP	A1	A2	P	beta
1:10642	1	10642	G	A	0.788298	0.00797757
1:11012	1	11012	G	C	0.917844	-0.00263639
1:13011	1	13011	T	G	0.787689	0.00831877
1:13110	1	13110	G	A	0.820159	0.00729131
1:13116	1	13116	T	G	0.871642	-0.00423368
1:13118	1	13118	G	A	0.814349	-0.007677
1:13284	1	13284	G	A	0.766499	-0.00964793
1:13289	1	13289	T	C	0.768181	-0.0095832
1:13380	1	13380	G	C	0.586043	0.017903


# Prepare AMP-PD data. Rename SNPs to chr:bp

Format AMP-PD files to have SNP names as chr:bp. This is still on the full AMP-PD data (not separating cohorts yet).

In [21]:
%%bash

cd /home/jupyter/pathways/

for PATHWAY in adaptive_immune alpha_synuclein innate_immune lysosome endocytosis mitochondria microglia monocytes
do

    #Make a new file to rename SNPs in the AMP-PD dataset - rename with just chr:bp
    awk '{ print $2"\t"$1":"$4}' "$PATHWAY".bim > "$PATHWAY"_chrbp.txt

    #Update names in plink
    /home/jupyter/plink2 --bfile /home/jupyter/pathways/"$PATHWAY" \
    --update-name /home/jupyter/pathways/"$PATHWAY"_chrbp.txt \
    --make-bed \
    --out /home/jupyter/pathways/"$PATHWAY".updated_names

    #Remove duplicate SNPs in plink
    /home/jupyter/plink2 --bfile /home/jupyter/pathways/"$PATHWAY".updated_names \
    --rm-dup force-first \
    --make-bed \
    --out /home/jupyter/pathways/"$PATHWAY".updated_names.no_dupli
    
done


PLINK v2.00a3LM AVX2 Intel (26 Aug 2021)       www.cog-genomics.org/plink/2.0/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/pathways/adaptive_immune.updated_names.log.
Options in effect:
  --bfile /home/jupyter/pathways/adaptive_immune
  --make-bed
  --out /home/jupyter/pathways/adaptive_immune.updated_names
  --update-name /home/jupyter/pathways/adaptive_immune_chrbp.txt

Start time: Thu Jun  9 13:27:33 2022
60292 MiB RAM detected; reserving 30146 MiB for main workspace.
Using up to 16 threads (change this with --threads).
2610 samples (951 females, 1659 males; 2610 founders) loaded from
/home/jupyter/pathways/adaptive_immune.fam.
108520 variants loaded from /home/jupyter/pathways/adaptive_immune.bim.
Note: No phenotype data present.
Writing /home/jupyter/pathways/adaptive_immune.updated_names.fam ... done.
Writing /home/jupyter/pathways/adaptive_immune.updated_names.bim ... done.
Writing /home/jupyter/pathways/adaptive_immune

# Run PRSice2: Merging by chr:bp. Loop over all pathways

Run PRSice2 without p-value optimisation, just make PRSs at set thresholds

The question is should we be merging the base and target datasets by SNP names including A1 and A2, or just chr:bp? PRSice says that strand flips are automatically detected and accounted for. I have tried both options and there are more SNPs overlapping between base and target datasets when using chr:bp

This is run in the full AMP-PD dataset so there are enough samples for clumping (recommended N > 500 - see https://www.prsice.info/command_detail/#clumping). If we separated out the cohorts before this stage, some cohorts would not be large enough for estimating LD for clumping internally. The PRSs are calculated independently for each sample and there are the same SNPs in the genetic data in each cohort. 

**Now using updated Chang2017 sumstats including 23andMe from Sara**

In [124]:
%%bash

for PATHWAY in adaptive_immune alpha_synuclein innate_immune lysosome endocytosis mitochondria microglia monocytes
do
    
    Rscript /home/jupyter/PRSice.R \
    --prsice /home/jupyter/PRSice_linux \
    --base /home/jupyter/sumstats/Chang2017_with23andMe_Sara/META4_hg38.tab \
    --pvalue P \
    --beta \
    --a1 A1 \
    --a2 A2 \
    --clump-kb 250 \
    --clump-r2 0.1 \
    --snp chrbp \
    --target /home/jupyter/pathways/"$PATHWAY".updated_names.no_dupli \
    --no-regress \
    --lower 5e-08 \
    --upper 0.05 \
    --interval 5e-08 \
    --print-snp \
    --out /home/jupyter/pathways/"$PATHWAY"_PD_PRS_Chang23andMe
done

Begin plotting
Current Rscript version = 2.3.3
Begin plotting
Current Rscript version = 2.3.3
Begin plotting
Current Rscript version = 2.3.3
Begin plotting
Current Rscript version = 2.3.3
Begin plotting
Current Rscript version = 2.3.3
Begin plotting
Current Rscript version = 2.3.3
Begin plotting
Current Rscript version = 2.3.3
Begin plotting
Current Rscript version = 2.3.3


PRSice 2.3.3 (2020-08-05) 
https://github.com/choishingwan/PRSice
(C) 2016-2020 Shing Wan (Sam) Choi and Paul F. O'Reilly
GNU General Public License v3
If you use PRSice in any published work, please cite:
Choi SW, O'Reilly PF.
PRSice-2: Polygenic Risk Score Software for Biobank-Scale Data.
GigaScience 8, no. 7 (July 1, 2019)
2023-04-25 12:31:29
/home/jupyter/PRSice_linux \
    --a1 A1 \
    --a2 A2 \
    --bar-levels 0.001,0.05,0.1,0.2,0.3,0.4,0.5,1 \
    --base /home/jupyter/sumstats/Chang2017_with23andMe_Sara/META4_hg38.tab \
    --beta  \
    --binary-target F \
    --bp BP \
    --chr CHR \
    --clump-kb 250kb \
    --clump-p 1.000000 \
    --clump-r2 0.100000 \
    --interval 5e-08 \
    --lower 5e-08 \
    --no-regress  \
    --num-auto 22 \
    --out /home/jupyter/pathways/adaptive_immune_PD_PRS_Chang23andMe \
    --print-snp  \
    --pvalue P \
    --seed 1709566672 \
    --snp chrbp \
    --stat BETA \
    --target /home/jupyter/pathways/adaptive_immune.updated_names.no_dupl

In [126]:
%%bash
cat /home/jupyter/pathways/lysosome_PD_PRS_Chang23andMe.snp

CHR	SNP	BP	P	Base
1	1:155235843	155235843	5.14e-15	1
1	1:155236376	155236376	2.735e-15	1
3	3:183133725	183133725	5.664e-15	1
4	4:987108	987108	6.143e-12	1
4	4:995234	995234	2.791e-09	1
4	4:76226816	76226816	2.137e-11	1
12	12:122941028	122941028	6.097e-09	1
17	17:42494785	42494785	1.534e-08	1
4	4:76182999	76182999	5.286e-07	1
4	4:1002080	1002080	2.456e-06	1
4	4:76163978	76163978	4.389e-06	1
14	14:87941544	87941544	1.075e-05	1
4	4:76222111	76222111	0.0001245	1
4	4:988557	988557	0.0001553	1
11	11:6616273	6616273	0.0001569	1
17	17:7580783	7580783	0.0001886	1
16	16:28479196	28479196	0.0003085	1
1	1:155236246	155236246	0.0003095	1
17	17:68319327	68319327	0.0003427	1
12	12:123735244	123735244	0.0004496	1
15	15:78924987	78924987	0.0005531	1
8	8:53748385	53748385	0.0005545	1
17	17:68421579	68421579	0.0005733	1
8	8:18070100	18070100	0.0007517	1
19	19:50359062	50359062	0.0007798	1
5	5:74721674	74721674	0.0008597	1
8	8:97852044	97852044	0.001007	1
4	4:155941958	155941958	0.001027	1
5	5:78112652	78

# Get cohort information

In [23]:
cohort_query = f"""

SELECT
participant_id, study
FROM `{BQ_RELEASE_DATASET}.amp_pd_participants`

"""

cohort = bq_query(cohort_query)

In [24]:
cohort.study.value_counts()

LBD        4586
PPMI       1945
PDBP       1606
HBS        1189
LCC         638
Steady      334
Sure        261
BioFIND     213
Name: study, dtype: int64

In [25]:
#Write full file as output
with open('/home/jupyter/clinical_data_v2-5/cohort.tab', 'w') as f:
    f.write(cohort.to_csv(index=False, sep='\t'))

In [26]:
cohorts = ["LBD", "PPMI", "PDBP", "HBS", "LCC", "Steady", "Sure", "BioFIND"]

#Loop over cohorts - select just individuals in each cohort and write as separate output
#This is so we can extract each cohort individuals from plink or PRSice2 files
for COHORT in cohorts:
    cohort_select = cohort.copy()
    cohort_select = cohort_select[cohort_select['study'] == COHORT]
    print(COHORT)
    
    with open(f'/home/jupyter/clinical_data_v2-5/{COHORT}.tab', 'w') as f:
        f.write(cohort_select.to_csv(index=False, sep='\t'))

LBD
PPMI
PDBP
HBS
LCC
Steady
Sure
BioFIND


In [27]:
%%bash
head /home/jupyter/clinical_data_v2-5/LBD.tab

participant_id	study
LB-00001	LBD
LB-00002	LBD
LB-00003	LBD
LB-00004	LBD
LB-00005	LBD
LB-00006	LBD
LB-00007	LBD
LB-00008	LBD
LB-00009	LBD


# Standardise PRSs within each cohort

Note that the final p-value threshold used depends on the number of SNPs - so it is not always labelled as Pt_0.05.

In [10]:
from sklearn.preprocessing import StandardScaler

cohorts = ["LBD", "PPMI", "PDBP", "HBS", "LCC", "Steady", "Sure", "BioFIND"]

pathways = ["adaptive_immune", "alpha_synuclein", "innate_immune", "lysosome", "endocytosis", "mitochondria", "microglia", "monocytes"]

for COHORT in cohorts:
    
    #Read in cohort file
    cohort = pd.read_csv(f'/home/jupyter/clinical_data_v2-5/{COHORT}.tab',
                         delim_whitespace=True)
    
    print(COHORT)
    
    for PATHWAY in pathways:
        
        #Read in PRS file
        PRS = pd.read_csv(f'/home/jupyter/pathways/{PATHWAY}_PD_PRS_Chang23andMe.all_score',
                     delim_whitespace=True)
        
        #Merge PRS with selected cohort - inner join
        merged = pd.merge(cohort, PRS, left_on='participant_id', right_on='FID', how = 'inner')
        
        #If there is no match and merged dataframe is empty - print warning
        #This will print multiple times - once for each pathway
        if merged.empty:
            print(f'{COHORT} dataFrame is empty!')
        
        else:
            PRS_edit = merged.copy()
        
            #The last column is either Pt_1 or Pt_0.05 (or closest threshold) as this depends on the number of SNPs included
            #If the last column is Pt_1, then remove this column, if not keep everything
            #This is because the columns are not consistently named - it depends on the SNPs in each pathway PRS
            if PRS_edit.columns[-1] == "Pt_1":
                PRS_edit = PRS_edit.drop("Pt_1", axis = 1)
            else :
                PRS_edit = PRS_edit

            #Select just relevant columns - PRS at p value 0.05 which is now the last column
            PRS_selected = PRS_edit.copy()
            PRS_selected = PRS_selected.iloc[:,[0,1,-1]]

            #Standardise the PRS - make new column with z-transformed PRS
            scaler = StandardScaler()
            PRS_selected['PRS_z'] = scaler.fit_transform(PRS_selected.iloc[:,[-1]])

            #Save as file
            with open('/home/jupyter/pathways/' + COHORT + '.' + PATHWAY + '_PRS_final.txt', 'w') as f:
                f.write(PRS_selected.to_csv(index=False, sep='\t'))


LBD
LBD dataFrame is empty!
LBD dataFrame is empty!
LBD dataFrame is empty!
LBD dataFrame is empty!
LBD dataFrame is empty!
LBD dataFrame is empty!
LBD dataFrame is empty!
LBD dataFrame is empty!
PPMI
PDBP
HBS
LCC
Steady
Sure
BioFIND


Make a results dataframe to store the final p threshold (the one closest to 0.05) and number of SNPs in the PRS for each pathway. This is the same for each cohort.

In [11]:
prsice_results = []

In [12]:
pathways = ["adaptive_immune", "alpha_synuclein", "innate_immune", "lysosome", "endocytosis", "mitochondria", "microglia", "monocytes"]

for index,pathway in enumerate(pathways):
    
    #Read in prsice file
    prsice = pd.read_csv(f'/home/jupyter/pathways/' + pathway + '_PD_PRS_Chang23andMe.prsice',
                 delim_whitespace=True)
    
    prsice_edit = prsice.copy()
    
    #The last row is either Pt_1 or Pt_0.05 (or closest threshold) as this depends on the number of SNPs included
    #If the last row threshold is 1, then remove this last row, if not keep everything
    if prsice_edit.iloc[-1,2] == 1.0 :
        prsice_edit = prsice_edit.drop(prsice_edit.index[-1], axis = 0)
    else :
        prsice_edit = prsice_edit
    
    #From the prsice results, get the PRS threshold and the number of SNPs
    list = prsice_edit.iloc[-1,2:4].values.tolist()
    
    #Add the pathway name to this list
    list.append(pathway)
    
    #Now join to the main results dataframe to make a list of lists
    prsice_results.append(list)

In [13]:
#Show the list of lists
prsice_results

[[0.04998, 733, 'adaptive_immune'],
 [0.04863, 75, 'alpha_synuclein'],
 [0.05, 1049, 'innate_immune'],
 [0.04966, 154, 'lysosome'],
 [0.04989, 350, 'endocytosis'],
 [0.04998, 1677, 'mitochondria'],
 [0.04995, 2001, 'microglia'],
 [0.04995, 1705, 'monocytes']]

In [14]:
#Convert list to dataframe
prsice_results_df = pd.DataFrame(prsice_results, columns = ['p_threshold_closest_to_0.05', 'n_snps', 'pathway'])

In [15]:
prsice_results_df

,p_threshold_closest_to_0.05,n_snps,pathway
0,0.04998,733,adaptive_immune
1,0.04863,75,alpha_synuclein
2,0.05000,1049,innate_immune
3,0.04966,154,lysosome
4,0.04989,350,endocytosis
5,0.04998,1677,mitochondria
6,0.04995,2001,microglia
7,0.04995,1705,monocytes


In [16]:
#Write as output
with open('/home/jupyter/pathways/n_snps_in_pathwayPRS_summary.tab', 'w') as f:
    f.write(prsice_results_df.to_csv(index=False, sep='\t'))

# Get age at diagnosis data

Query clinical data

In [23]:
age_dx_query = f"""

SELECT 
*
FROM `{BQ_RELEASE_DATASET}.PD_Medical_History`

"""

age_dx = bq_query(age_dx_query)

In [24]:
#Check STEADY-PD participants - they seem to be all missing age at diagnosis
age_dx[age_dx.participant_id.str.contains("SY")]

,participant_id,GUID,visit_name,visit_month,diagnosis,initial_diagnosis,most_recent_diagnosis,change_in_diagnosis,change_in_diagnosis_months_after_baseline,surgery_for_parkinson_disease,pd_diagnosis_months_after_baseline,age_at_diagnosis,pd_medication_initiation_months_after_baseline,pd_medication_start_months_after_baseline,use_of_pd_medication,pd_medication_recent_use_months_after_baseline,on_levodopa,on_dopamine_agonist,on_other_pd_medications,diagnosis_type
6936,SY-NIH_INVAA791MKCET,NIH_INVAA791MKCET,M0,0.0,Parkinson's Disease,None,None,None,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None
6937,SY-NIH_INVAA976PRYBT,NIH_INVAA976PRYBT,M0,0.0,Parkinson's Disease,None,None,None,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None
6938,SY-NIH_INVBC118NRWC4,NIH_INVBC118NRWC4,M0,0.0,Parkinson's Disease,None,None,None,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None
6939,SY-NIH_INVCW212TXMFX,NIH_INVCW212TXMFX,M0,0.0,Parkinson's Disease,None,None,None,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None
6940,SY-NIH_INVCX656ULCDX,NIH_INVCX656ULCDX,M0,0.0,Parkinson's Disease,None,None,None,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24079,SY-PDXZ650DHD,PDXZ650DHD,M42,42.0,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,None,NaN,Yes,None,No,None
24080,SY-PDZJ450XYU,PDZJ450XYU,M42,42.0,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,None,NaN,Yes,None,No,None
24081,SY-PDZN008XD9,PDZN008XD9,M42,42.0,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,None,NaN,Yes,None,No,None
24082,SY-PDZP715EGW,PDZP715EGW,M42,42.0,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,None,NaN,Yes,None,No,None


In [25]:
age_dx.head()

,participant_id,GUID,visit_name,visit_month,diagnosis,initial_diagnosis,most_recent_diagnosis,change_in_diagnosis,change_in_diagnosis_months_after_baseline,surgery_for_parkinson_disease,pd_diagnosis_months_after_baseline,age_at_diagnosis,pd_medication_initiation_months_after_baseline,pd_medication_start_months_after_baseline,use_of_pd_medication,pd_medication_recent_use_months_after_baseline,on_levodopa,on_dopamine_agonist,on_other_pd_medications,diagnosis_type
0,BF-1002,PDCB969UGG,LOG,NaN,None,None,None,None,NaN,None,NaN,NaN,NaN,-39.0,None,NaN,None,None,None,None
1,BF-1003,PDLW805AHT,LOG,NaN,None,None,None,None,NaN,None,NaN,NaN,NaN,-57.0,None,NaN,None,None,None,None
2,BF-1004,PDKW284DYW,LOG,NaN,None,None,None,None,NaN,None,NaN,NaN,NaN,-76.0,None,NaN,None,None,None,None
3,BF-1006,PDKY484YDC,LOG,NaN,None,None,None,None,NaN,None,NaN,NaN,NaN,-13.0,None,NaN,None,None,None,None
4,BF-1008,PDEA056CRM,LOG,NaN,None,None,None,None,NaN,None,NaN,NaN,NaN,-74.0,None,NaN,None,None,None,None


In [26]:
age_dx[age_dx.participant_id == "BF-1002"]

,participant_id,GUID,visit_name,visit_month,diagnosis,initial_diagnosis,most_recent_diagnosis,change_in_diagnosis,change_in_diagnosis_months_after_baseline,surgery_for_parkinson_disease,pd_diagnosis_months_after_baseline,age_at_diagnosis,pd_medication_initiation_months_after_baseline,pd_medication_start_months_after_baseline,use_of_pd_medication,pd_medication_recent_use_months_after_baseline,on_levodopa,on_dopamine_agonist,on_other_pd_medications,diagnosis_type
0,BF-1002,PDCB969UGG,LOG,NaN,None,None,None,None,NaN,None,NaN,NaN,NaN,-39.0,None,NaN,None,None,None,None
9190,BF-1002,PDCB969UGG,M0,0.0,Idiopathic PD,None,None,None,NaN,None,-61.0,61.0,NaN,NaN,Yes,0.0,Yes,No,No,None


Going to use the age of diagnosis variable. There used to be a date of diagnosis variable in v2.0 but this has been removed in v2.5 of the dataset.

Note this data has multiple rows per participant as data is recorded at each visit. Want to remove duplicates but take just the most frequently reported age at diagnosis.

In [27]:
#First remove rows missing age at diagnosis
age_dx_nomiss = age_dx[age_dx['age_at_diagnosis'].notnull()]

In [28]:
age_dx_nomiss.head()

,participant_id,GUID,visit_name,visit_month,diagnosis,initial_diagnosis,most_recent_diagnosis,change_in_diagnosis,change_in_diagnosis_months_after_baseline,surgery_for_parkinson_disease,pd_diagnosis_months_after_baseline,age_at_diagnosis,pd_medication_initiation_months_after_baseline,pd_medication_start_months_after_baseline,use_of_pd_medication,pd_medication_recent_use_months_after_baseline,on_levodopa,on_dopamine_agonist,on_other_pd_medications,diagnosis_type
5685,LC-1000006,None,M0,0.0,Parkinson's Disease,None,None,None,NaN,None,-108.0,63.0,NaN,NaN,None,NaN,None,None,None,None
5686,LC-100001,None,M0,0.0,Parkinson's Disease,None,None,None,NaN,None,-36.0,71.0,NaN,NaN,None,NaN,None,None,None,None
5689,LC-100005,None,M0,0.0,Parkinson's Disease,None,None,None,NaN,None,-108.0,52.0,NaN,NaN,None,NaN,None,None,None,None
5690,LC-100006,None,M0,0.0,Parkinson's Disease,None,None,None,NaN,None,-264.0,50.0,NaN,NaN,None,NaN,None,None,None,None
5691,LC-100008,None,M0,0.0,Parkinson's Disease,None,None,None,NaN,None,-84.0,51.0,NaN,NaN,None,NaN,None,None,None,None


Note there are some individuals who report different age at diagnosis at different visits. Here we want to take the most frequently reported age at diagnosis

In [29]:
#Now get the most frequent age at diagnosis reported for each participant (the mode)
age_dx_nomiss_nodupli = age_dx_nomiss.groupby('participant_id').age_at_diagnosis.agg(lambda x : x.mode()[0])

In [30]:
age_dx_nomiss_nodupli.head()

participant_id
BF-1002    61.0
BF-1003    56.0
BF-1004    55.0
BF-1006    52.0
BF-1008    65.0
Name: age_at_diagnosis, dtype: float64

Extract just PD cases (PD at study entry and latest diagnosis, excluding known genetic PD cases). This is the same list of individuals we have extracted from the genetic data

In [31]:
PDcases_nongenetic = pd.read_csv(f'/home/jupyter/clinical_data_v2-5/PD_cases_start_end_dx_nongenetic.tab',
                 delimiter="\t")

PDcases_agedx = pd.merge(PDcases_nongenetic, age_dx_nomiss_nodupli, left_on='FID', right_on='participant_id', how = 'left')

PDcases_agedx.head()

,FID,IID,age_at_diagnosis
0,BF-1002,BF-1002,61.0
1,BF-1003,BF-1003,56.0
2,BF-1004,BF-1004,55.0
3,BF-1006,BF-1006,52.0
4,BF-1008,BF-1008,65.0


Check if there are any cases missing age at diagnosis

In [32]:
PDcases_agedx[PDcases_agedx['age_at_diagnosis'].isnull()]

,FID,IID,age_at_diagnosis
1281,LC-130003,LC-130003,NaN
1284,LC-180003,LC-180003,NaN
1285,LC-190003,LC-190003,NaN
1293,LC-280003,LC-280003,NaN
1297,LC-340003,LC-340003,NaN
...,...,...,...
2831,SY-PDZW544CEL,SY-PDZW544CEL,NaN
2832,SY-PDZX554VNB,SY-PDZX554VNB,NaN
2833,SY-PDZX943HWN,SY-PDZX943HWN,NaN
2834,SY-PDZY968RFA,SY-PDZY968RFA,NaN


In [33]:
#Write as output
with open('/home/jupyter/clinical_data_v2-5/PD_cases_agedx.tab', 'w') as f:
    f.write(PDcases_agedx.to_csv(index=False, sep='\t'))

# Generate genetic principal components to include as covariates

Generate genetic PCs in each cohort separately. Excluding LBD as we excluded these samples from the genetic dataset.

First prune for independent SNPs.

In [5]:
%%bash

for COHORT in PPMI PDBP HBS LCC Steady Sure BioFIND
do
/home/jupyter/plink2 --bfile /home/jupyter/genetic_data_v2-5/merged.PDonly.updated_sex.mind_0.05.het.binary.xsplit.nongenetic.no_dupli.auto_snps.european.pihat.geno.missing_hap.hwe.maf \
                --keep-fam /home/jupyter/clinical_data_v2-5/"$COHORT".tab \
                --indep-pairwise 50 5 0.01 \
                --out /home/jupyter/genetic_data_v2-5/"$COHORT".pruning \
                --threads 16
done

PLINK v2.00a3LM AVX2 Intel (26 Aug 2021)       www.cog-genomics.org/plink/2.0/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/genetic_data_v2-5/PPMI.pruning.log.
Options in effect:
  --bfile /home/jupyter/genetic_data_v2-5/merged.PDonly.updated_sex.mind_0.05.het.binary.xsplit.nongenetic.no_dupli.auto_snps.european.pihat.geno.missing_hap.hwe.maf
  --indep-pairwise 50 5 0.01
  --keep-fam /home/jupyter/clinical_data_v2-5/PPMI.tab
  --out /home/jupyter/genetic_data_v2-5/PPMI.pruning
  --threads 16

Start time: Thu Jun 23 11:46:49 2022
60292 MiB RAM detected; reserving 30146 MiB for main workspace.
Using up to 16 threads (change this with --threads).
2610 samples (951 females, 1659 males; 2610 founders) loaded from
/home/jupyter/genetic_data_v2-5/merged.PDonly.updated_sex.mind_0.05.het.binary.xsplit.nongenetic.no_dupli.auto_snps.european.pihat.geno.missing_hap.hwe.maf.fam.
7737986 variants loaded from
/home/jupyter/genetic_data_v2-5/m

Extract just independent SNPs

In [6]:
%%bash

for COHORT in PPMI PDBP HBS LCC Steady Sure BioFIND
do
/home/jupyter/plink2 --bfile /home/jupyter/genetic_data_v2-5/merged.PDonly.updated_sex.mind_0.05.het.binary.xsplit.nongenetic.no_dupli.auto_snps.european.pihat.geno.missing_hap.hwe.maf \
                --keep-fam /home/jupyter/clinical_data_v2-5/"$COHORT".tab \
                --extract /home/jupyter/genetic_data_v2-5/"$COHORT".pruning.prune.in \
                --make-bed \
                --out /home/jupyter/genetic_data_v2-5/"$COHORT".pruned \
                --threads 16
done

PLINK v2.00a3LM AVX2 Intel (26 Aug 2021)       www.cog-genomics.org/plink/2.0/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/genetic_data_v2-5/PPMI.pruned.log.
Options in effect:
  --bfile /home/jupyter/genetic_data_v2-5/merged.PDonly.updated_sex.mind_0.05.het.binary.xsplit.nongenetic.no_dupli.auto_snps.european.pihat.geno.missing_hap.hwe.maf
  --extract /home/jupyter/genetic_data_v2-5/PPMI.pruning.prune.in
  --keep-fam /home/jupyter/clinical_data_v2-5/PPMI.tab
  --make-bed
  --out /home/jupyter/genetic_data_v2-5/PPMI.pruned
  --threads 16

Start time: Thu Jun 23 11:49:03 2022
60292 MiB RAM detected; reserving 30146 MiB for main workspace.
Using up to 16 threads (change this with --threads).
2610 samples (951 females, 1659 males; 2610 founders) loaded from
/home/jupyter/genetic_data_v2-5/merged.PDonly.updated_sex.mind_0.05.het.binary.xsplit.nongenetic.no_dupli.auto_snps.european.pihat.geno.missing_hap.hwe.maf.fam.
7737986 varian

Perform PCA on independent SNPs

In [7]:
%%bash

for COHORT in PPMI PDBP HBS LCC Steady Sure BioFIND
do
/home/jupyter/plink2 --bfile /home/jupyter/genetic_data_v2-5/"$COHORT".pruned \
                --pca \
                --out /home/jupyter/genetic_data_v2-5/"$COHORT".pca \
                --threads 16
done

PLINK v2.00a3LM AVX2 Intel (26 Aug 2021)       www.cog-genomics.org/plink/2.0/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/genetic_data_v2-5/PPMI.pca.log.
Options in effect:
  --bfile /home/jupyter/genetic_data_v2-5/PPMI.pruned
  --out /home/jupyter/genetic_data_v2-5/PPMI.pca
  --pca
  --threads 16

Start time: Thu Jun 23 11:49:35 2022
60292 MiB RAM detected; reserving 30146 MiB for main workspace.
Using up to 16 threads (change this with --threads).
465 samples (165 females, 300 males; 465 founders) loaded from
/home/jupyter/genetic_data_v2-5/PPMI.pruned.fam.
162495 variants loaded from /home/jupyter/genetic_data_v2-5/PPMI.pruned.bim.
Note: No phenotype data present.
Calculating allele frequencies... 0%40%80%done.
Constructing GRM: 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%

# Prepare for R analysis

In [11]:
%load_ext rpy2.ipython

In [79]:
%%bash
mkdir -p /home/jupyter/R_packages
#Make directory R_packages if it does not already exist

In [80]:
%%R
pack <- "/home/jupyter/R_packages"

#COMMENT OUT PACKAGES ALREADY INSTALLED

#install.packages("survival",lib = pack)
#install.packages("survminer", lib = pack)
#install.packages("dplyr", lib = pack)
#install.packages("data.table", lib = pack)
#install.packages("tidyr", lib = pack)
#install.packages("ggpubr", lib = pack) 
#install.packages("ggtext", lib = pack)
#install.packages("lme4", lib = pack)
#install.packages("ggplot2", lib = pack)
#install.packages("lmerTest", lib = pack)
#install.packages("tidyr", lib = pack)

In [12]:
%%R
pack <- "/home/jupyter/R_packages"
library(dplyr, lib.loc = pack)
library(ggplot2, lib.loc = pack)
library(lme4, lib.loc = pack)
library(lmerTest, lib.loc = pack)
library(tidyr, lib.loc = pack)
library(ggpubr, lib.loc = pack)
library(survival,lib.loc = pack)
library(survminer, lib.loc = pack)

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


R[write to console]: 
Attaching package: ‘ggplot2’


R[write to console]: The following object is masked _by_ ‘.GlobalEnv’:

    expand_scale


R[write to console]: Loading required package: Matrix

R[write to console]: 
Attaching package: ‘lmerTest’


R[write to console]: The following object is masked from ‘package:lme4’:

    lmer


R[write to console]: The following object is masked from ‘package:stats’:

    step


R[write to console]: 
Attaching package: ‘tidyr’


R[write to console]: The following objects are masked from ‘package:Matrix’:

    expand, pack, unpack


R[write to console]: 
Attaching package: ‘survminer’


R[write to console]: The following object is masked from ‘package:survival’:

    myeloma




# Analyse AAO vs pathways in R

In [20]:
%%R

#Make list of pathways
pathways <- c("adaptive_immune", "alpha_synuclein", "innate_immune", "lysosome", "endocytosis",
              "microglia", "monocytes", "mitochondria")

#Make list of cohorts - excluding LBD as these samples have been excluded from the genetic dataset
cohorts <- c("PPMI", "PDBP", "HBS", "LCC", "Steady", "Sure", "BioFIND")

#Make function to analyse
analyse_aao <- function(pathway, cohort) {
    
        #Read in data for PRS, age at diagnosis, cohort, and genetic PCs
        PRS <- read.table(paste("/home/jupyter/pathways/", cohort, ".", pathway, "_PRS_final.txt", sep = ""), 
                          sep = "\t", header = T)

        aao <- read.table("/home/jupyter/clinical_data_v2-5/PD_cases_agedx.tab", sep = "\t", header = T)

        demo <- read.table("/home/jupyter/clinical_data_v2-5/PD_cases_start_end_dx_sex.tab", sep = "\t", header = T)

        #Read in genetic PCs
        pcs <- read.table(paste("/home/jupyter/genetic_data_v2-5/", cohort, ".pca.eigenvec", sep = ""))
        colnames(pcs) <- c("FID", "IID", paste("PC", 1:10, sep = ""))

        #Merge
        merged <- PRS %>%
        inner_join(aao, by = c("participant_id" = "IID")) %>%
        inner_join(demo, by = c("participant_id" = "IID")) %>%
        inner_join(pcs, by = c("participant_id" = "IID"))
        
        #Select just relevant columns
        merged_selected <- merged %>%
            select(participant_id, study, PRS_z, sex, age_at_diagnosis, PC1:PC5)
        
        #Remove individuals missing any rows
        merged_selected_nomiss <- merged_selected %>%
            drop_na()
    
        N <- dim(merged_selected_nomiss)[1]
                
        #If the final dataframe is not empty, run GLM
        if (dim(merged_selected_nomiss)[1] != 0) {
            
            #Run GLM
            glm_AAO_PRS <- glm(age_at_diagnosis ~ PRS_z + sex + PC1 + PC2 + PC3 + PC4 + PC5,
                          data = merged_selected_nomiss)
            
            #Get summary from coefficients
            summary <- c(pathway,
                        cohort,
                        summary(glm_AAO_PRS)$coefficients[2,1],
                        summary(glm_AAO_PRS)$coefficients[2,2],
                        summary(glm_AAO_PRS)$coefficients[2,4],
                        N)
            
        } else {
            
            #If the dataframe is empty, print NA for summary stat
            summary <- c(pathway,
                        cohort,
                        rep(NA, 3),
                        N)
            }
    
    return(summary)
        
}

In [21]:
%%R

results_list <- list()

#Loop over each pathway and cohort
for (pathway in pathways) {
    for (cohort in cohorts) {
        i <- length(results_list)
        results_list[[i+1]] <- analyse_aao(pathway, cohort)
    }
}

#Combine lists into a single dataframe
aao_results <- do.call(rbind, results_list)

#Set column names
colnames(aao_results) <- c("pathway", "cohort", "Coeff", "se", "Pvalue", "N")

#Make into dataframe
aao_results <- as.data.frame(aao_results)

#Save output
write.table(aao_results, "/home/jupyter/pathways/pathways_agediagnosis_results.txt",
           sep = "\t", row.names = F, col.names = T, quote = F)

aao_results

           pathway  cohort                Coeff                se
1  adaptive_immune    PPMI     0.11754008310212 0.446742353872611
2  adaptive_immune    PDBP   -0.332335186986961 0.367177480640906
3  adaptive_immune     HBS   -0.261082261339774 0.424091577967974
4  adaptive_immune     LCC   -0.317705278196143  1.17640874019523
5  adaptive_immune  Steady                 <NA>              <NA>
6  adaptive_immune    Sure    0.432857079106381 0.616280311399835
7  adaptive_immune BioFIND     0.65651521882082 0.681548832826391
8  alpha_synuclein    PPMI   -0.895683264513345 0.445336129686749
9  alpha_synuclein    PDBP   -0.224143381442031 0.364606977516569
10 alpha_synuclein     HBS   -0.470040328419086 0.427452936204207
11 alpha_synuclein     LCC     0.81491485331913  1.23696805918159
12 alpha_synuclein  Steady                 <NA>              <NA>
13 alpha_synuclein    Sure   -0.919593274652152 0.606858978170641
14 alpha_synuclein BioFIND    0.545375716464445 0.678939189025328
15   innat

# Get cohort demographics for TABLE 1

Count the number of individuals in each cohort - after genetic QC, only individuals who have at least sex and age at diagnosis. The number of individuals who have available outcomes may be less.

In [36]:
%%bash

head /home/jupyter/clinical_data_v2-5/cohort.tab

participant_id	study
BF-1001	BioFIND
BF-1002	BioFIND
BF-1003	BioFIND
BF-1004	BioFIND
BF-1005	BioFIND
BF-1006	BioFIND
BF-1008	BioFIND
BF-1009	BioFIND
BF-1010	BioFIND


In [37]:
%%R

#Read in one of the pathway fam files - can be any one as they should all be the same
fam <- read.table("/home/jupyter/pathways/adaptive_immune.fam")
colnames(fam) <- c("FID", "IID", "m", "f", "sex_fam", "pheno")

#Read in AAO data
aao <- read.table("/home/jupyter/clinical_data_v2-5/PD_cases_agedx.tab", sep = "\t", header = T)

#Read in demographic data
demo <- read.table("/home/jupyter/clinical_data_v2-5/PD_cases_start_end_dx_sex.tab", sep = "\t", header = T)

#Read in cohort data
cohort <- read.table("/home/jupyter/clinical_data_v2-5/cohort.tab", header = T)

#Merge
merged <- fam %>%
    inner_join(aao, by = c("FID", "IID")) %>%
    inner_join(demo, by = c("FID", "IID")) %>%
    inner_join(cohort, by = c("IID" = "participant_id"))

print(dim(merged))

[1] 2610   10


In [38]:
%%R

#Looks like all the Steady-PD cohort are missing age at diagnosis - have checked against the raw data 
merged %>%
filter(study == "Steady")

                     FID                  IID m f sex_fam pheno
1   SY-NIH_INVAA791MKCET SY-NIH_INVAA791MKCET 0 0       1    -9
2   SY-NIH_INVAA976PRYBT SY-NIH_INVAA976PRYBT 0 0       1    -9
3   SY-NIH_INVAL031EKGJ6 SY-NIH_INVAL031EKGJ6 0 0       1    -9
4   SY-NIH_INVAN212WJYBM SY-NIH_INVAN212WJYBM 0 0       1    -9
5   SY-NIH_INVBB604RTZN4 SY-NIH_INVBB604RTZN4 0 0       1    -9
6   SY-NIH_INVBC118NRWC4 SY-NIH_INVBC118NRWC4 0 0       1    -9
7   SY-NIH_INVBP682BYGHF SY-NIH_INVBP682BYGHF 0 0       1    -9
8   SY-NIH_INVCB901YLVWG SY-NIH_INVCB901YLVWG 0 0       1    -9
9   SY-NIH_INVCW212TXMFX SY-NIH_INVCW212TXMFX 0 0       2    -9
10  SY-NIH_INVCX656ULCDX SY-NIH_INVCX656ULCDX 0 0       1    -9
11  SY-NIH_INVDE149YELY9 SY-NIH_INVDE149YELY9 0 0       1    -9
12  SY-NIH_INVEF231BNLJU SY-NIH_INVEF231BNLJU 0 0       1    -9
13  SY-NIH_INVEG915UKFH3 SY-NIH_INVEG915UKFH3 0 0       1    -9
14  SY-NIH_INVEP886EEYYL SY-NIH_INVEP886EEYYL 0 0       1    -9
15  SY-NIH_INVEY006JDFDF SY-NIH_INVEY006

In [41]:
%%R
head(merged)

      FID     IID m f sex_fam pheno age_at_diagnosis sex age_at_baseline
1 BF-1002 BF-1002 0 0       2    -9               61   2              66
2 BF-1003 BF-1003 0 0       1    -9               56   1              61
3 BF-1004 BF-1004 0 0       1    -9               55   1              62
4 BF-1006 BF-1006 0 0       1    -9               52   1              59
5 BF-1008 BF-1008 0 0       2    -9               65   2              69
6 BF-1010 BF-1010 0 0       1    -9               58   1              65
    study
1 BioFIND
2 BioFIND
3 BioFIND
4 BioFIND
5 BioFIND
6 BioFIND


In [42]:
%%R

#Get final cohort demographics for TABLE 1 - after genetic QC, only individuals who have at least sex and age at diagnosis
merged %>%
group_by(study) %>%
filter(!is.na(sex)) %>%
filter(!is.na(age_at_diagnosis)) %>%
summarise(count = n(),
          mean_aad = mean(age_at_diagnosis),
          sd_aad = sd(age_at_diagnosis),
          mean_entry = mean(age_at_baseline),
          sd_entry = sd(age_at_baseline))

# A tibble: 6 × 6
  study   count mean_aad sd_aad mean_entry sd_entry
  <chr>   <int>    <dbl>  <dbl>      <dbl>    <dbl>
1 BioFIND    92     60.9   6.20       67.7     6.26
2 HBS       616     61.9  10.5        66.0     9.97
3 LCC        64     60.2   9.31       68.0     9.67
4 PDBP      790     58.7  10.2        64.5     9.02
5 PPMI      465     61.3   9.65       61.8     9.71
6 Sure      247     62.0   9.52       62.7     9.50


In [43]:
%%R

#Check total number of individuals (not separated by cohort) that have the key data
merged %>%
filter(!is.na(sex)) %>%
filter(!is.na(age_at_diagnosis)) %>%
summarise(count = n())

  count
1  2274


In [44]:
%%R

#Get counts of men vs. women in each cohort
merged %>%
filter(!is.na(sex)) %>%
filter(!is.na(age_at_diagnosis)) %>%
group_by(study, sex) %>%
summarise(count = n())

`summarise()` has grouped output by 'study'. You can override using the `.groups` argument.
# A tibble: 12 × 3
# Groups:   study [6]
   study     sex count
   <chr>   <int> <int>
 1 BioFIND     1    59
 2 BioFIND     2    33
 3 HBS         1   395
 4 HBS         2   221
 5 LCC         1    42
 6 LCC         2    22
 7 PDBP        1   503
 8 PDBP        2   287
 9 PPMI        1   300
10 PPMI        2   165
11 Sure        1   127
12 Sure        2   120


# Analyse pathways vs MDS-UPDRSIII

Query MDS-UPDRS Part III data, including Hoehn and Yahr stage

In [58]:
mdsupdrs3_query = f"""

SELECT 
participant_id, visit_name, visit_month, mds_updrs_part_iii_summary_score, upd2hy_hoehn_and_yahr_stage
FROM `{BQ_RELEASE_DATASET}.MDS_UPDRS_Part_III`

"""

mdsupdrs3 = bq_query(mdsupdrs3_query)

In [59]:
mdsupdrs3 = mdsupdrs3.sort_values(by = ['participant_id', 'visit_month'])
mdsupdrs3

,participant_id,visit_name,visit_month,mds_updrs_part_iii_summary_score,upd2hy_hoehn_and_yahr_stage
854,BF-1001,M0,0.0,0.0,Stage 0
2239,BF-1002,M0,0.0,15.0,Stage 1
21518,BF-1002,M0_5,0.5,16.0,Stage 1
2240,BF-1003,M0,0.0,12.0,Stage 1
21525,BF-1003,M0_5,0.5,24.0,Stage 2
...,...,...,...,...,...
17382,SY-PDZX943HWN,M36,36.0,10.0,Stage 2
2663,SY-PDZY968RFA,M0,0.0,11.0,Stage 1
16964,SY-PDZY968RFA,M36,36.0,22.0,Stage 2
3089,SY-PDZZ260HUM,M0,0.0,13.0,Stage 2


In [60]:
#Look at values of visit month
mdsupdrs3.visit_month.value_counts()
#Could still include negative visit month values

 0.0      5047
 12.0     2426
 24.0     2227
 6.0      1953
 36.0     1826
 18.0     1718
 30.0     1454
 48.0     1135
-1.0       988
 60.0      869
 42.0      795
 84.0      527
 3.0       519
 72.0      497
 54.0      475
 9.0       463
 96.0      266
 0.5       118
 108.0      42
-2.0         8
Name: visit_month, dtype: int64

In [61]:
#Check for duplicated entries by participant id and visit month
mdsupdrs3[mdsupdrs3.duplicated(['participant_id', 'visit_month'], keep = False)]

,participant_id,visit_name,visit_month,mds_updrs_part_iii_summary_score,upd2hy_hoehn_and_yahr_stage
19051,PP-3001,M48,48.0,39.0,Stage 2
22585,PP-3001,M48#2,48.0,35.0,Stage 2
20241,PP-3001,M60,60.0,34.0,Stage 2
22840,PP-3001,M60#2,60.0,26.0,Stage 2
21089,PP-3001,M84,84.0,36.0,Stage 2
...,...,...,...,...,...
21691,PP-75524,M12#2,12.0,8.0,Stage 1
4811,PP-75562,M0,0.0,38.0,Stage 3
21511,PP-75562,M0#2,0.0,17.0,Stage 2
11096,PP-75562,M12,12.0,41.0,Stage 2


In [62]:
#There are some duplicates with a second entry at the same visit (with #2 in the visit name)
#Remove rows with #
mdsupdrs3_nodupli = mdsupdrs3.copy()
mdsupdrs3_nodupli = mdsupdrs3_nodupli[~mdsupdrs3_nodupli.visit_name.str.contains('#')]

In [63]:
#Check for any other duplicated entries by participant id and visit month
mdsupdrs3_nodupli[mdsupdrs3_nodupli.duplicated(['participant_id', 'visit_month'], keep = False)]

,participant_id,visit_name,visit_month,mds_updrs_part_iii_summary_score,upd2hy_hoehn_and_yahr_stage


In [64]:
#Write as output
with open('/home/jupyter/clinical_data_v2-5/mdsupdrs3_allparticipants.tab', 'w') as f:
    f.write(mdsupdrs3_nodupli.to_csv(index=False, sep='\t'))

In [9]:
%%R

#Make list of pathways
pathways <- c("adaptive_immune", "alpha_synuclein", "innate_immune", "lysosome", "endocytosis",
              "microglia", "monocytes", "mitochondria")

#Make list of cohorts - excluding LBD as these samples have been excluded from the genetic dataset
cohorts <- c("PPMI", "PDBP", "HBS", "LCC", "Steady", "Sure", "BioFIND")

#Make function to analyse longitudinal outcomes using mixed effects models
#df is the name of the .tab file in /home/jupyter/clinical_data_v2-5/ which has the data in long format
#df = "mdsupdrs3_allparticipants"
#outcome is the variable name of the outcome within df
#outcome = "mds_updrs_part_ii_summary_score"

analyse_cont <- function(pathway, cohort, df, outcome) {
    
        #Read in data for PRS, age at diagnosis, cohort, and genetic PCs
        PRS <- read.table(paste("/home/jupyter/pathways/", cohort, ".", pathway, "_PRS_final.txt", sep = ""), 
                          sep = "\t", header = T)
    
        data <- read.table(paste("/home/jupyter/clinical_data_v2-5/", df, ".tab", sep = ""), 
                             sep = "\t", header = T)
    
        aao <- read.table("/home/jupyter/clinical_data_v2-5/PD_cases_agedx.tab", sep = "\t", header = T)

        demo <- read.table("/home/jupyter/clinical_data_v2-5/PD_cases_start_end_dx_sex.tab", sep = "\t", header = T)

        #Read in genetic PCs
        pcs <- read.table(paste("/home/jupyter/genetic_data_v2-5/", cohort, ".pca.eigenvec", sep = ""))
        colnames(pcs) <- c("FID", "IID", paste("PC", 1:10, sep = ""))

        #Merge - in long format, starting from the longitudinal outcome data
        merged <- data %>%
        inner_join(PRS, by = "participant_id") %>%
        inner_join(aao, by = c("participant_id" = "FID")) %>%
        inner_join(demo, by = "IID") %>%
        inner_join(pcs, by = c("FID", "IID"))

        #Select the outcome variable from the function argument
        merged$y <- merged[[outcome]]
    
        #Select just relevant columns
        #This cannot be done with dplyr::select
        merged_selected <- merged[, c('y', 'participant_id', 'study', 'PRS_z', 'sex', 
                                      'age_at_baseline', 'age_at_diagnosis', 'visit_month',
                                      'PC1', 'PC2', 'PC3', 'PC4', 'PC5')]
    
        #Remove individuals missing any rows
        merged_selected_nomiss <- merged_selected %>%
            drop_na()
    
        #Get mean and SD of outcome at baseline
        print(merged_selected_nomiss %>% 
            filter(visit_month == 0) %>%
            summarise(mean = mean(y),
                     sd = sd(y)))
    
        #Make variable years from diagnosis
        merged_selected_nomiss <- merged_selected_nomiss %>%
        mutate(disease_duration_at_entry = age_at_baseline - age_at_diagnosis) %>%
        mutate(years_from_diagnosis = disease_duration_at_entry + visit_month/12)
        
        #Count total number of observations
        N_obs <- dim(merged_selected_nomiss)[1]
        
        #Count total number of individuals - removing duplicated rows by participant_id
        uniq_id <- merged_selected_nomiss %>%
        distinct(participant_id)
            
        N_inds <- dim(uniq_id)[1]

        #For mixed effects models, we usually need 2 x number of individuals because we are estimating random effects for both the intercept and slope
        #If the number of observations is more than 2 times the number of individuals (this is because we have N_inds * 2 random effects, one for the intercept and one for the slope)
        if (N_obs > N_inds*2) {
            
            #Run GLM with random intercept and slope per individual
            lmer_PRS <- lmer(y ~ PRS_z + PRS_z*years_from_diagnosis
                                + sex + sex*years_from_diagnosis
                                + age_at_diagnosis
                                + PC1 + PC2 + PC3 + PC4 + PC5
                                + (years_from_diagnosis | participant_id),
                                data = merged_selected_nomiss,
                                REML = TRUE)

            
            #Save model coefficients to results table
            #Want the slope coefficient PRS x time from diagnosis
            summary <- c(pathway,
                         cohort,
                        summary(lmer_PRS)$coefficients['PRS_z:years_from_diagnosis',1],
                        summary(lmer_PRS)$coefficients['PRS_z:years_from_diagnosis',2],
                        summary(lmer_PRS)$coefficients['PRS_z:years_from_diagnosis',3],
                        summary(lmer_PRS)$coefficients['PRS_z:years_from_diagnosis',4],
                        summary(lmer_PRS)$coefficients['PRS_z:years_from_diagnosis',5],
                        N_obs,
                        N_inds)
            
            } else if (N_obs > N_inds) {
            
            #If there are not enough observations to fit both random intercept and slope (N_obs < 2*N_inds)
            #Run GLM with random intercept only
            lmer_PRS <- lmer(y ~ PRS_z + PRS_z*years_from_diagnosis
                                + sex + sex*years_from_diagnosis
                                + age_at_diagnosis
                                + PC1 + PC2 + PC3 + PC4 + PC5
                                + (1 | participant_id),
                                data = merged_selected_nomiss,
                                REML = TRUE)

            
            #Save model coefficients to results table
            #Want the slope coefficient PRS x time from diagnosis
            summary <- c(pathway,
                         cohort,
                        summary(lmer_PRS)$coefficients['PRS_z:years_from_diagnosis',1],
                        summary(lmer_PRS)$coefficients['PRS_z:years_from_diagnosis',2],
                        summary(lmer_PRS)$coefficients['PRS_z:years_from_diagnosis',3],
                        summary(lmer_PRS)$coefficients['PRS_z:years_from_diagnosis',4],
                        summary(lmer_PRS)$coefficients['PRS_z:years_from_diagnosis',5],
                        N_obs,
                        N_inds)
            
            } else {
            
        
            #If the dataframe is empty, print NA for summary stat
            summary <- c(pathway,
                        cohort,
                        rep(NA, 5),
                        N_obs,
                        N_inds)
            
            }
        
                     
    return(summary)
        
}

In [11]:
%%R

updrs3_results_list <- list()

#Loop over each pathway and cohort
for (pathway in pathways) {
    for (cohort in cohorts) {
        print(cohort)
        i <- length(updrs3_results_list)
        updrs3_results_list[[i+1]] <- analyse_cont(pathway, cohort, "mdsupdrs3_allparticipants", "mds_updrs_part_iii_summary_score")
    }
}

[1] "PPMI"
      mean       sd
1 20.04662 9.259633
[1] "PDBP"
      mean       sd
1 25.46819 13.48902
[1] "HBS"
  mean sd
1  NaN NA
[1] "LCC"
      mean       sd
1 24.66667 15.01111
[1] "Steady"
  mean sd
1  NaN NA
[1] "Sure"
      mean       sd
1 22.27935 9.121966
[1] "BioFIND"
      mean      sd
1 28.26087 14.3893
[1] "PPMI"
      mean       sd
1 20.04662 9.259633
[1] "PDBP"
      mean       sd
1 25.46819 13.48902
[1] "HBS"
  mean sd
1  NaN NA
[1] "LCC"
      mean       sd
1 24.66667 15.01111
[1] "Steady"
  mean sd
1  NaN NA
[1] "Sure"
      mean       sd
1 22.27935 9.121966
[1] "BioFIND"
      mean      sd
1 28.26087 14.3893
[1] "PPMI"
      mean       sd
1 20.04662 9.259633
[1] "PDBP"
      mean       sd
1 25.46819 13.48902
[1] "HBS"
  mean sd
1  NaN NA
[1] "LCC"
      mean       sd
1 24.66667 15.01111
[1] "Steady"
  mean sd
1  NaN NA
[1] "Sure"
      mean       sd
1 22.27935 9.121966
[1] "BioFIND"
      mean      sd
1 28.26087 14.3893
[1] "PPMI"
      mean       sd
1 20.04662 9.25

In [24]:
%%R

#Combine lists into a single dataframe
updrs3_results <- do.call(rbind, updrs3_results_list)

#Set column names
colnames(updrs3_results) <- c("pathway", "cohort", "Coeff", "se", "df", "t_value", "Pvalue", "N_obs", "N_inds")

#Make into dataframe
updrs3_results <- as.data.frame(updrs3_results)

#Save output
write.table(updrs3_results, "/home/jupyter/pathways/pathways_mdsupdrs3_results.txt",
           sep = "\t", row.names = F, col.names = T, quote = F)

updrs3_results

           pathway  cohort               Coeff                 se
1  adaptive_immune    PPMI  0.0721015112189424  0.102865127038644
2  adaptive_immune    PDBP   0.204767173955193 0.0919642819515198
3  adaptive_immune     HBS                <NA>               <NA>
4  adaptive_immune     LCC                <NA>               <NA>
5  adaptive_immune  Steady                <NA>               <NA>
6  adaptive_immune    Sure   0.294400922273215  0.235417361481696
7  adaptive_immune BioFIND   0.417312692183422  0.392696002966269
8  alpha_synuclein    PPMI  0.0693572600950412  0.107291291622245
9  alpha_synuclein    PDBP  0.0537062026147313 0.0935439218821116
10 alpha_synuclein     HBS                <NA>               <NA>
11 alpha_synuclein     LCC                <NA>               <NA>
12 alpha_synuclein  Steady                <NA>               <NA>
13 alpha_synuclein    Sure   0.116111593194773  0.234422621200671
14 alpha_synuclein BioFIND   0.246137505035106  0.369339441828035
15   innat

# Analyse pathways vs MDS-UPDRSII

In [135]:
mdsupdrs2_query = f"""

SELECT 
participant_id, visit_name, visit_month, mds_updrs_part_ii_summary_score
FROM `{BQ_RELEASE_DATASET}.MDS_UPDRS_Part_II`

"""

mdsupdrs2 = bq_query(mdsupdrs2_query)

In [136]:
mdsupdrs2 = mdsupdrs2.sort_values(by = ['participant_id', 'visit_month'])
mdsupdrs2

,participant_id,visit_name,visit_month,mds_updrs_part_ii_summary_score
3335,BF-1002,M0,0.0,11.0
1149,BF-1003,M0,0.0,11.0
4220,BF-1004,M0,0.0,11.0
3624,BF-1006,M0,0.0,14.0
1400,BF-1008,M0,0.0,3.0
...,...,...,...,...
16062,SY-PDZX943HWN,M36,36.0,5.0
3333,SY-PDZY968RFA,M0,0.0,0.0
16300,SY-PDZY968RFA,M36,36.0,5.0
3334,SY-PDZZ260HUM,M0,0.0,1.0


In [137]:
#Check for duplicated entries by participant id and visit month
mdsupdrs2[mdsupdrs2.duplicated(['participant_id', 'visit_month'], keep = False)]

,participant_id,visit_name,visit_month,mds_updrs_part_ii_summary_score
11398,PP-3066,M18,18.0,13.0
11399,PP-3066,M18#2,18.0,15.0
11109,PP-3067,M18,18.0,14.0
11570,PP-3067,M18#2,18.0,9.0
11217,PP-3078,M18,18.0,5.0
11402,PP-3078,M18#2,18.0,7.0
4970,PP-3251,M3,3.0,7.0
5109,PP-3251,M3#2,3.0,11.0
5273,PP-3252,M3#2,3.0,9.0
5274,PP-3252,M3,3.0,9.0


In [138]:
#Remove duplicated visit entries - rows with #
mdsupdrs2_nodupli = mdsupdrs2.copy()
mdsupdrs2_nodupli = mdsupdrs2_nodupli[~mdsupdrs2_nodupli.visit_name.str.contains('#')]

In [139]:
#Check for any other duplicated entries by participant id and visit month
mdsupdrs2_nodupli[mdsupdrs2_nodupli.duplicated(['participant_id', 'visit_month'], keep = False)]

,participant_id,visit_name,visit_month,mds_updrs_part_ii_summary_score


In [140]:
#Write as output
with open('/home/jupyter/clinical_data_v2-5/mdsupdrs2_allparticipants.tab', 'w') as f:
    f.write(mdsupdrs2_nodupli.to_csv(index=False, sep='\t'))

In [12]:
%%R

updrs2_results_list <- list()

#Loop over each pathway and cohort
for (pathway in pathways) {
    for (cohort in cohorts) {
        print(cohort)
        i <- length(updrs2_results_list)
        updrs2_results_list[[i+1]] <- analyse_cont(pathway, cohort, "mdsupdrs2_allparticipants", "mds_updrs_part_ii_summary_score")
    }
}

[1] "PPMI"
      mean       sd
1 5.771028 4.138572
[1] "PDBP"
      mean       sd
1 10.79625 7.767168
[1] "HBS"
  mean sd
1  NaN NA
[1] "LCC"
      mean       sd
1 11.66667 13.42882
[1] "Steady"
  mean sd
1  NaN NA
[1] "Sure"
      mean       sd
1 5.789474 4.316283
[1] "BioFIND"
      mean      sd
1 11.04348 6.35906
[1] "PPMI"
      mean       sd
1 5.771028 4.138572
[1] "PDBP"
      mean       sd
1 10.79625 7.767168
[1] "HBS"
  mean sd
1  NaN NA
[1] "LCC"
      mean       sd
1 11.66667 13.42882
[1] "Steady"
  mean sd
1  NaN NA
[1] "Sure"
      mean       sd
1 5.789474 4.316283
[1] "BioFIND"
      mean      sd
1 11.04348 6.35906
[1] "PPMI"
      mean       sd
1 5.771028 4.138572
[1] "PDBP"
      mean       sd
1 10.79625 7.767168
[1] "HBS"
  mean sd
1  NaN NA
[1] "LCC"
      mean       sd
1 11.66667 13.42882
[1] "Steady"
  mean sd
1  NaN NA
[1] "Sure"
      mean       sd
1 5.789474 4.316283
[1] "BioFIND"
      mean      sd
1 11.04348 6.35906
[1] "PPMI"
      mean       sd
1 5.771028 4.13

In [26]:
%%R

#Combine lists into a single dataframe
updrs2_results <- do.call(rbind, updrs2_results_list)

#Set column names
colnames(updrs2_results) <- c("pathway", "cohort", "Coeff", "se", "df", "t_value", "Pvalue", "N_obs", "N_inds")

#Make into dataframe
updrs2_results <- as.data.frame(updrs2_results)

#Save output
write.table(updrs2_results, "/home/jupyter/pathways/pathways_mdsupdrs2_results.txt",
           sep = "\t", row.names = F, col.names = T, quote = F)

updrs2_results

           pathway  cohort               Coeff                 se
1  adaptive_immune    PPMI  0.0435532363926763 0.0524325288887569
2  adaptive_immune    PDBP  0.0692447472877802 0.0516395557084703
3  adaptive_immune     HBS                <NA>               <NA>
4  adaptive_immune     LCC                <NA>               <NA>
5  adaptive_immune  Steady                <NA>               <NA>
6  adaptive_immune    Sure  0.0400043398960736 0.0983470886906584
7  adaptive_immune BioFIND                <NA>               <NA>
8  alpha_synuclein    PPMI 0.00244144009967074 0.0546595102967342
9  alpha_synuclein    PDBP  0.0494776409269386 0.0526424916445889
10 alpha_synuclein     HBS                <NA>               <NA>
11 alpha_synuclein     LCC                <NA>               <NA>
12 alpha_synuclein  Steady                <NA>               <NA>
13 alpha_synuclein    Sure  0.0483134294830268 0.0980916166758742
14 alpha_synuclein BioFIND                <NA>               <NA>
15   innat

# Analyse pathways vs H&Y3+

In [115]:
#Read in H&Y data (in MDS-UPDRS dataset)
mdsupdrs3_nodupli = pd.read_csv(f'/home/jupyter/notebooks/clinical_data_v2-5/mdsupdrs3_allparticipants.tab',
                               delimiter = "\t")

#Read in age at diagnosis and demographics data
demo = pd.read_csv(f'/home/jupyter/notebooks/clinical_data_v2-5/PD_cases_start_end_dx_sex.tab',
                    delimiter = "\t")
    
aao = pd.read_csv(f'/home/jupyter/notebooks/clinical_data_v2-5/PD_cases_agedx.tab',
                   delimiter = "\t").loc[:, ['FID', 'IID', 'age_at_diagnosis']]

In [116]:
#Merge with demographics and age at diagnosis data to get time from diagnosis
temp = mdsupdrs3_nodupli.merge(demo, left_on = 'participant_id', right_on = 'IID')
merged = temp.merge(aao, on = (['FID', 'IID']))

In [117]:
#Calculate time from diagnosis for each visit
merged['YEARS_DIAGNOSIS_TO_BL'] = merged.age_at_baseline - merged.age_at_diagnosis
merged['YEARS_FROM_DIAGNOSIS'] = merged.YEARS_DIAGNOSIS_TO_BL + merged.visit_month/12

In [118]:
#Make new variable for H&Y stage as numeric
merged['HOEHN_YAHR_STAGE'] = merged.upd2hy_hoehn_and_yahr_stage.str.replace(r'Stage ', '').astype(float)

#Make new variable coding whether H&Y stage is >=3 at each visit
merged['HY3'] = [np.nan if pd.isna(i)
                else 1 if i >=3
                else 0 for i in merged.HOEHN_YAHR_STAGE]

In [119]:
#Make a variable for H&Y3 EVER  - whether a patient ever has a HY3 greater than one in the course of their study 
HYEV = merged.loc[:, ["participant_id", 'HY3']].groupby("participant_id").agg(func=max)

#Join to main HY dataframe 
HY_merged = merged.join(HYEV, on = "participant_id", rsuffix="_EV").reset_index(drop = True)

In [120]:
HY_merged

,participant_id,visit_name,visit_month,mds_updrs_part_iii_summary_score,upd2hy_hoehn_and_yahr_stage,FID,IID,sex,age_at_baseline,age_at_diagnosis,YEARS_DIAGNOSIS_TO_BL,YEARS_FROM_DIAGNOSIS,HOEHN_YAHR_STAGE,HY3,HY3_EV
0,BF-1002,M0,0.0,15.0,Stage 1,BF-1002,BF-1002,2,66,61.0,5.0,5.000000,1.0,0.0,0.0
1,BF-1002,M0_5,0.5,16.0,Stage 1,BF-1002,BF-1002,2,66,61.0,5.0,5.041667,1.0,0.0,0.0
2,BF-1003,M0,0.0,12.0,Stage 1,BF-1003,BF-1003,1,61,56.0,5.0,5.000000,1.0,0.0,0.0
3,BF-1003,M0_5,0.5,24.0,Stage 2,BF-1003,BF-1003,1,61,56.0,5.0,5.041667,2.0,0.0,0.0
4,BF-1004,M0,0.0,33.0,Stage 2,BF-1004,BF-1004,1,62,55.0,7.0,7.000000,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14427,SY-PDZX943HWN,M36,36.0,10.0,Stage 2,SY-PDZX943HWN,SY-PDZX943HWN,1,46,NaN,NaN,NaN,2.0,0.0,0.0
14428,SY-PDZY968RFA,M0,0.0,11.0,Stage 1,SY-PDZY968RFA,SY-PDZY968RFA,1,41,NaN,NaN,NaN,1.0,0.0,0.0
14429,SY-PDZY968RFA,M36,36.0,22.0,Stage 2,SY-PDZY968RFA,SY-PDZY968RFA,1,41,NaN,NaN,NaN,2.0,0.0,0.0
14430,SY-PDZZ260HUM,M0,0.0,13.0,Stage 2,SY-PDZZ260HUM,SY-PDZZ260HUM,2,70,NaN,NaN,NaN,2.0,0.0,0.0


In [121]:
#Now want to calculate the time to event for H&Y3+

#First filter for individuals who do not ever meet H&Y3+, and take the last visit age where H&Y3+ was not met
HY3_no = (HY_merged[(HY_merged.HY3_EV == 0) & (HY_merged.HY3 == 0)].drop_duplicates(subset=["participant_id"],keep="last")
                               .rename({"YEARS_FROM_DIAGNOSIS":"HY3_TIME_FROM_DIAGNOSIS"}, axis=1))

#Secondly filter for individuals who have meet H&Y3+, and take the FIRST visit age where H&Y3+ was met
#Here need to filter for H&Y3+ EVER as well as H&Y3 at each visit
HY3_yes = (HY_merged[(HY_merged.HY3_EV == 1) & (HY_merged.HY3 == 1)].drop_duplicates(subset=["participant_id"],keep="first")
                               .rename({"YEARS_FROM_DIAGNOSIS":"HY3_TIME_FROM_DIAGNOSIS"}, axis=1))

In [122]:
#Join dataframes - concatenate (add rows)
HY3 = pd.concat([HY3_no, HY3_yes]).reset_index(drop = True)

#Check for duplicates
HY3_dup = HY3[HY3.duplicated(['participant_id'], keep = False)]
n_dup = len(HY3_dup)
print(str(n_dup) + ' duplicated')

#If a patient has H&Y3+ at baseline (visit_month == 0), they are left censored - code as 2 
HY3.loc[:,"HY3_EV"] = HY3.HY3_EV.mask((HY3.visit_month == 0) & (HY3.HY3 == 1), 2)

0 duplicated


In [123]:
#Now select relevant columns - only one row per participant
HY3_export = HY3.copy()
HY3_export = HY3_export.loc[:, ['FID', 'IID', 'age_at_diagnosis', 'age_at_baseline', 'sex', 'HY3_EV', 'HY3_TIME_FROM_DIAGNOSIS']]

In [124]:
#Write as output
#Note this is all the participants with clinical data, not just our list of PD only participants
with open('/home/jupyter/notebooks/clinical_data_v2-5/HY3_allparticipants.tab', 'w') as f:
    f.write(HY3_export.to_csv(index=False, sep='\t'))

In [125]:
HY3_export.head()

,FID,IID,age_at_diagnosis,age_at_baseline,sex,HY3_EV,HY3_TIME_FROM_DIAGNOSIS
0,BF-1002,BF-1002,61.0,66,2,0.0,5.041667
1,BF-1003,BF-1003,56.0,61,1,0.0,5.041667
2,BF-1004,BF-1004,55.0,62,1,0.0,7.041667
3,BF-1006,BF-1006,52.0,59,1,0.0,7.041667
4,BF-1008,BF-1008,65.0,69,2,0.0,4.041667


Now run survival analysis in R

In [27]:
%%R
pack <- "/home/jupyter/notebooks/R_packages"
library(dplyr, lib.loc = pack)
library(ggplot2, lib.loc = pack)
library(ggpubr, lib.loc = pack)
library(survival,lib.loc = pack)
library(survminer, lib.loc = pack)

In [28]:
%%R

#Make list of pathways
pathways <- c("adaptive_immune", "alpha_synuclein", "innate_immune", "lysosome", "endocytosis",
              "microglia", "monocytes", "mitochondria")

#Make list of cohorts - excluding LBD as these samples have been excluded from the genetic dataset
cohorts <- c("PPMI", "PDBP", "HBS", "LCC", "Steady", "Sure", "BioFIND")

#Make function to analyse longitudinal binomial outcomes using survival Cox proportional hazards models
#df is the name of the .tab file in /home/jupyter/clinical_data_v2-5/ which has the data in long format
#df = "HY3_allparticipants"
#outcome is the variable name of the outcome within df
#outcome = "HY3"

analyse_surv <- function(pathway, cohort, df, outcome) {
    
        #Read in data for PRS, clinical data, and genetic PCs
        PRS <- read.table(paste("/home/jupyter/pathways/", cohort, ".", pathway, "_PRS_final.txt", sep = ""), 
                          sep = "\t", header = T)
    
        data <- read.table(paste("/home/jupyter/clinical_data_v2-5/", df, ".tab", sep = ""), 
                             sep = "\t", header = T)
    
        #Read in genetic PCs
        pcs <- read.table(paste("/home/jupyter/genetic_data_v2-5/", cohort, ".pca.eigenvec", sep = ""))
        colnames(pcs) <- c("FID", "IID", paste("PC", 1:10, sep = ""))

        #Merge
        merged <- PRS %>%
        inner_join(data, by = c("participant_id" = "IID")) %>%
        inner_join(pcs, by = c("FID"))
        
        #Select the outcome variable from the function argument
        merged$event <- merged[[paste(outcome, "_EV", sep ="")]]
    
        #Select the time to event variable from the function argument
        merged$time_to_event <- merged[[paste(outcome, "_TIME_FROM_DIAGNOSIS", sep ="")]]

        #Filter out individuals who met outcome at baseline (left censored, coded as 2)
        #Otherwise the Surv function to make the survival object misinterprets the codes
        merged <- merged %>%
            filter(event <= 1)
        
        #Select just relevant columns
        #This cannot be done with dplyr::select
        merged_selected <- merged[, c('event', 'time_to_event', 'participant_id', 'study', 'PRS_z', 'sex', 
                                      'age_at_diagnosis',
                                      'PC1', 'PC2', 'PC3', 'PC4', 'PC5')]
    
        #Remove individuals missing any rows
        merged_selected_nomiss <- merged_selected %>%
            drop_na()
    
        #Number of individuals in dataset
        N_inds <- dim(merged_selected_nomiss)[1]
    
        #If < 20 individuals have met the outcome of interest
        if (sum(merged_selected_nomiss$event == 1) > 20) {
            
            #Run Cox proportional hazards analysis for HY3 adjusting for age at diagnosis and gender
            surv_object_HY3 <- Surv(time = merged_selected_nomiss$time_to_event, event = merged_selected_nomiss$event)
    
            #Fit Cox Proportional Hazards model with covariates age at diagnosis and gender and PC1-5
            model.cox_HY3 = coxph(surv_object_HY3 ~ PRS_z + age_at_diagnosis + sex 
                              + PC1 + PC2 + PC3 + PC4 + PC5, 
                              data = merged_selected_nomiss)
    
            kmz <- cox.zph(model.cox_HY3, transform = "km")
    
            #Save model coefficients to results table
            summary <- c(pathway,
                         cohort,
                         summary(model.cox_HY3)$coefficients[1,1],
                         summary(model.cox_HY3)$coefficients[1,3],
                         summary(model.cox_HY3)$coefficients[1,5],
                         kmz$table[1,3],
                         summary(model.cox_HY3)$logtest[[1]],
                         summary(model.cox_HY3)$sctest[[1]],
                         summary(model.cox_HY3)$rsq[[1]],
                         N_inds) # nagelkerke r square

    
        } else {
            
            #If not enough individuals met the outcome or the dataframe is empty, print NA for summary stat
            summary <- c(pathway,
                        cohort,
                        rep(NA, 7),
                        N_inds)
        }
    
    return(summary)

}

In [29]:
%%R

HY3_results_list <- list()

#Loop over each pathway and cohort
for (pathway in pathways) {
    for (cohort in cohorts) {
        i <- length(HY3_results_list)
        HY3_results_list[[i+1]] <- analyse_surv(pathway, cohort, "HY3_allparticipants", "HY3")
    }
}

In [31]:
%%R

#Combine lists into a single dataframe
HY3_results <- do.call(rbind, HY3_results_list)

#Set column names
colnames(HY3_results) <- c("pathway", "study", "Coeff", "se", "Pvalue", "Cox.zphPVal", "ov.lik.ratio","logrank", "r2", "N")

#Make into dataframe
HY3_results <- as.data.frame(HY3_results)

#Save output
write.table(HY3_results, "/home/jupyter/pathways/pathways_HY3_results.txt",
           sep = "\t", row.names = F, col.names = T, quote = F)

HY3_results

           pathway   study               Coeff                 se
1  adaptive_immune    PPMI   0.100677662295459 0.0946301767819111
2  adaptive_immune    PDBP   0.143502378099667  0.125561194430677
3  adaptive_immune     HBS                <NA>               <NA>
4  adaptive_immune     LCC                <NA>               <NA>
5  adaptive_immune  Steady                <NA>               <NA>
6  adaptive_immune    Sure                <NA>               <NA>
7  adaptive_immune BioFIND                <NA>               <NA>
8  alpha_synuclein    PPMI -0.0536558363959514   0.10663081240186
9  alpha_synuclein    PDBP  -0.164685029248054   0.13746711273961
10 alpha_synuclein     HBS                <NA>               <NA>
11 alpha_synuclein     LCC                <NA>               <NA>
12 alpha_synuclein  Steady                <NA>               <NA>
13 alpha_synuclein    Sure                <NA>               <NA>
14 alpha_synuclein BioFIND                <NA>               <NA>
15   innat

# Analyse pathways vs MoCA

Query MoCA data

In [48]:
moca_query = f"""

SELECT 
*
FROM `{BQ_RELEASE_DATASET}.MOCA`
"""
moca = bq_query(moca_query)

In [49]:
moca.head()

,participant_id,GUID,visit_name,visit_month,moca01_alternating_trail_making,moca02_visuoconstr_skills_cube,moca03_visuoconstr_skills_clock_cont,moca04_visuoconstr_skills_clock_num,moca05_visuoconstr_skills_clock_hands,moca_visuospatial_executive_subscore,...,moca22_orientation_date_score,moca23_orientation_month_score,moca24_orientation_year_score,moca25_orientation_day_score,moca26_orientation_place_score,moca27_orientation_city_score,moca_orientation_subscore,code_education_12years_complete,education_12years_complete,moca_total_score
0,LC-100001,None,M0,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,24
1,LC-100001,None,M12,12.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,23
2,LC-100001,None,M24,24.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,18
3,LC-100001,None,M36,36.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,22
4,LC-10080001,None,M0,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,28


Note that there is no data on individual items, so will just use the total score - assume this has already been adjusted for education

In [50]:
moca = moca.sort_values(by = ['participant_id', 'visit_month'])
moca = moca.loc[:, ['participant_id', 'visit_name', 'visit_month', 'moca_total_score']]

In [51]:
moca.head()

,participant_id,visit_name,visit_month,moca_total_score
4706,BF-1001,M0,0.0,28
8735,BF-1002,M0,0.0,29
10591,BF-1003,M0,0.0,30
7216,BF-1004,M0,0.0,28
4707,BF-1005,M0,0.0,27


In [52]:
#Remove duplicated visit entries - rows with #
moca_nodupli = moca.copy()
moca_nodupli = moca_nodupli[~moca_nodupli.visit_name.str.contains('#')]

In [53]:
#Check for any other duplicated entries by participant id and visit month
moca_nodupli[moca_nodupli.duplicated(['participant_id', 'visit_month'], keep = False)]

,participant_id,visit_name,visit_month,moca_total_score
10349,SU-32242,M24,24.0,28
13069,SU-32242,M24,24.0,29
3829,SU-33107,M24,24.0,25
5496,SU-33107,M24,24.0,27
5497,SU-33113,M24,24.0,29
13288,SU-33113,M24,24.0,30
13290,SU-33114,M24,24.0,26
13291,SU-33114,M24,24.0,30
10490,SY-NIH_INVLD826LLAN9,M0,0.0,27
10491,SY-NIH_INVLD826LLAN9,M0,0.0,29


In [54]:
#There are still a few individuals with duplicated entries by participant_id and visit_month with different MoCA scores
#Just keep the first datapoint
moca_nodupli_final = moca_nodupli.copy()
moca_nodupli_final = moca_nodupli_final.drop_duplicates(subset = ['participant_id', 'visit_month'], keep = 'first')

In [55]:
moca_nodupli_final.head()

,participant_id,visit_name,visit_month,moca_total_score
4706,BF-1001,M0,0.0,28
8735,BF-1002,M0,0.0,29
10591,BF-1003,M0,0.0,30
7216,BF-1004,M0,0.0,28
4707,BF-1005,M0,0.0,27


In [56]:
#Write as output
with open('/home/jupyter/clinical_data_v2-5/moca_allparticipants.tab', 'w') as f:
    f.write(moca_nodupli_final.to_csv(index=False, sep='\t'))

In [32]:
%%R

moca_results_list <- list()

#Loop over each pathway and cohort
for (pathway in pathways) {
    for (cohort in cohorts) {
        i <- length(moca_results_list)
        moca_results_list[[i+1]] <- analyse_cont(pathway, cohort, "moca_allparticipants", "moca_total_score")

    }
}

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular



In [33]:
%%R

#Combine lists into a single dataframe
moca_results <- do.call(rbind, moca_results_list)

#Set column names
colnames(moca_results) <- c("pathway", "cohort", "Coeff", "se", "df", "t_value", "Pvalue", "N_obs", "N_inds")

#Make into dataframe
moca_results <- as.data.frame(moca_results)

#Save output
write.table(moca_results, "/home/jupyter/pathways/pathways_moca_results.txt",
           sep = "\t", row.names = F, col.names = T, quote = F)

moca_results

           pathway  cohort                Coeff                 se
1  adaptive_immune    PPMI -0.00331208833891141 0.0283583864000862
2  adaptive_immune    PDBP  -0.0374590940531551 0.0235467372857831
3  adaptive_immune     HBS                 <NA>               <NA>
4  adaptive_immune     LCC    0.027228585154695  0.101372211103009
5  adaptive_immune  Steady                 <NA>               <NA>
6  adaptive_immune    Sure  -0.0573211717939767 0.0490121898218669
7  adaptive_immune BioFIND                 <NA>               <NA>
8  alpha_synuclein    PPMI -0.00130287689163538 0.0296109535366347
9  alpha_synuclein    PDBP  -0.0247545864977931 0.0238875290784027
10 alpha_synuclein     HBS                 <NA>               <NA>
11 alpha_synuclein     LCC  -0.0490108368325559  0.096440187026592
12 alpha_synuclein  Steady                 <NA>               <NA>
13 alpha_synuclein    Sure  -0.0346465700283331 0.0483450228982943
14 alpha_synuclein BioFIND                 <NA>               

# Analyse pathways vs dementia

In [88]:
MOCA_cutoff = 21
MMSE_cutoff = 26
#These cutoffs should be inclusive - i.e. MoCA <=21 and MMSE <= 26

Query MMSE data

In [90]:
mmse_query = f"""

SELECT
participant_id, visit_name, visit_month, mms112_total_score
FROM `{BQ_RELEASE_DATASET}.MMSE_corrected_20220506`

"""
mmse = bq_query(mmse_query)

In [91]:
mmse = mmse.sort_values(by = ['participant_id', 'visit_month'])

In [92]:
mmse.head()

,participant_id,visit_name,visit_month,mms112_total_score
124,HB-PD_INVAA223GY7,M0,0.0,27
380,HB-PD_INVAA769TBF,M0,0.0,29
701,HB-PD_INVAB109VHC,M0,0.0,30
702,HB-PD_INVAB289LG3,M0,0.0,30
559,HB-PD_INVAB465GYE,M0,0.0,29


In [93]:
#Remove duplicated visit entries - rows with #
mmse_nodupli = mmse.copy()
mmse_nodupli = mmse_nodupli[~mmse_nodupli.visit_name.str.contains('#')]

In [94]:
#Check for any other duplicated entries by participant id and visit month
mmse_nodupli[mmse_nodupli.duplicated(['participant_id', 'visit_month'], keep = False)]

,participant_id,visit_name,visit_month,mms112_total_score


In [95]:
#Write as output
with open('/home/jupyter/clinical_data_v2-5/mmse_allparticipants.tab', 'w') as f:
    f.write(mmse_nodupli.to_csv(index=False, sep='\t'))

In [96]:
#Read in MoCA data
moca_nodupli = pd.read_csv('/home/jupyter/clinical_data_v2-5/moca_allparticipants.tab',
                          sep = '\t')
moca_nodupli.head()

,participant_id,visit_name,visit_month,moca_total_score
0,BF-1001,M0,0.0,28.0
1,BF-1002,M0,0.0,29.0
2,BF-1003,M0,0.0,30.0
3,BF-1004,M0,0.0,28.0
4,BF-1005,M0,0.0,27.0


In [97]:
#Check whether there are individuals who have done both the MoCA and MMSE
both_moca_mmse = moca_nodupli.merge(mmse_nodupli, on = ['participant_id', 'visit_name'])
both_moca_mmse

,participant_id,visit_name,visit_month_x,moca_total_score,visit_month_y,mms112_total_score
0,PD-PDAA955TCY,M0,0.0,25.0,0.0,29
1,PD-PDAJ440URR,M0,0.0,29.0,0.0,30
2,PD-PDBK615WC6,M0,0.0,27.0,0.0,30
3,PD-PDBY238FWE,M0,0.0,22.0,0.0,28
4,PD-PDCF053TPC,M0,0.0,28.0,0.0,29
5,PD-PDCR944LPD,M0,0.0,28.0,0.0,29
6,PD-PDDC904HWE,M0,0.0,28.0,0.0,30
7,PD-PDDF170BDZ,M0,0.0,27.0,0.0,30
8,PD-PDED633MRU,M0,0.0,25.0,0.0,28
9,PD-PDEH654NL2,M0,0.0,27.0,0.0,30


In [98]:
#Merge all individuals by participant_id and visit_name
moca_mmse = moca_nodupli.merge(mmse_nodupli, on = ['participant_id', 'visit_name', 'visit_month'], how = 'outer')
moca_mmse.head()

,participant_id,visit_name,visit_month,moca_total_score,mms112_total_score
0,BF-1001,M0,0.0,28.0,<NA>
1,BF-1002,M0,0.0,29.0,<NA>
2,BF-1003,M0,0.0,30.0,<NA>
3,BF-1004,M0,0.0,28.0,<NA>
4,BF-1005,M0,0.0,27.0,<NA>


In [99]:
#Read in age at diagnosis and demographics data
demo = pd.read_csv(f'/home/jupyter/clinical_data_v2-5/PD_cases_start_end_dx_sex.tab',
                    delimiter = "\t")

aao = pd.read_csv(f'/home/jupyter/clinical_data_v2-5/PD_cases_agedx.tab',
                   delimiter = "\t").loc[:, ['FID', 'IID', 'age_at_diagnosis']]

In [100]:
#Merge with demographics and age at diagnosis data to get time from diagnosis
temp = moca_mmse.merge(demo, left_on = 'participant_id', right_on = 'IID')
merged = temp.merge(aao, on = (['FID', 'IID']))

In [101]:
merged.head()

,participant_id,visit_name,visit_month,moca_total_score,mms112_total_score,FID,IID,sex,age_at_baseline,age_at_diagnosis
0,BF-1002,M0,0.0,29.0,<NA>,BF-1002,BF-1002,2,66,61.0
1,BF-1003,M0,0.0,30.0,<NA>,BF-1003,BF-1003,1,61,56.0
2,BF-1004,M0,0.0,28.0,<NA>,BF-1004,BF-1004,1,62,55.0
3,BF-1006,M0,0.0,25.0,<NA>,BF-1006,BF-1006,1,59,52.0
4,BF-1008,M0,0.0,29.0,<NA>,BF-1008,BF-1008,2,69,65.0


In [102]:
#Check whether there are any individuals with multiple entires for visit_name and visit_month
#Because we merged on participant_id + visit_name + visit_month, there may be some individuals with the same visit name but different visit months
merged[merged.duplicated(['participant_id', 'visit_name'])]

,participant_id,visit_name,visit_month,moca_total_score,mms112_total_score,FID,IID,sex,age_at_baseline,age_at_diagnosis


In [103]:
#Calculate time from diagnosis for each visit
merged['YEARS_DIAGNOSIS_TO_BL'] = merged.age_at_baseline - merged.age_at_diagnosis
merged['YEARS_FROM_DIAGNOSIS'] = merged.YEARS_DIAGNOSIS_TO_BL + merged.visit_month/12

In [104]:
#Make new variable coding whether MoCA <=21 (or whatever predefined cutoff) at each visit
merged['MOCA21'] = [np.nan if pd.isna(i)
                else 1 if i <= MOCA_cutoff
                else 0 for i in merged.moca_total_score]

merged['MMSE26'] = [np.nan if pd.isna(i)
                else 1 if i <= MMSE_cutoff
                else 0 for i in merged.mms112_total_score]

#Make variable which is the sum of the MOCA and MMSE cutoff variables
merged['MOCA_MMSE_cutoffs'] = merged.fillna(0)['MOCA21'] + merged.fillna(0)['MMSE26']

#Now make a combined variable which codes dementia if EITHER MOCA or MMSE are below cutoffs
merged['dementia'] = [np.nan if pd.isna(i)
                else 1 if i >= 1
                else 0 for i in merged.MOCA_MMSE_cutoffs]

In [105]:
merged[merged.dementia == 1]

,participant_id,visit_name,visit_month,moca_total_score,mms112_total_score,FID,IID,sex,age_at_baseline,age_at_diagnosis,YEARS_DIAGNOSIS_TO_BL,YEARS_FROM_DIAGNOSIS,MOCA21,MMSE26,MOCA_MMSE_cutoffs,dementia
10,BF-1032,M0,0.0,18.0,<NA>,BF-1032,BF-1032,1,70,63.0,7.0,7.0,1.0,NaN,1.0,1
88,BF-1177,M0,0.0,20.0,<NA>,BF-1177,BF-1177,1,73,62.0,11.0,11.0,1.0,NaN,1.0,1
100,BF-1210,M0,0.0,18.0,<NA>,BF-1210,BF-1210,1,73,65.0,8.0,8.0,1.0,NaN,1.0,1
127,LC-1490001,M0,0.0,21.0,<NA>,LC-1490001,LC-1490001,2,71,64.0,7.0,7.0,1.0,NaN,1.0,1
129,LC-1490001,M36,36.0,20.0,<NA>,LC-1490001,LC-1490001,2,71,64.0,7.0,10.0,1.0,NaN,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7427,HB-PD_INVYU764JYK,M0,0.0,NaN,24,HB-PD_INVYU764JYK,HB-PD_INVYU764JYK,1,87,80.0,7.0,7.0,NaN,1.0,1.0,1
7440,HB-PD_INVZF591WW3,M0,0.0,NaN,25,HB-PD_INVZF591WW3,HB-PD_INVZF591WW3,2,69,65.0,4.0,4.0,NaN,1.0,1.0,1
7443,HB-PD_INVZJ012MP6,M0,0.0,NaN,26,HB-PD_INVZJ012MP6,HB-PD_INVZJ012MP6,2,64,60.0,4.0,4.0,NaN,1.0,1.0,1
7454,HB-PD_INVZW010GB2,M0,0.0,NaN,13,HB-PD_INVZW010GB2,HB-PD_INVZW010GB2,1,75,65.0,10.0,10.0,NaN,1.0,1.0,1


In [111]:
#Make a variable for dementia EVER  - whether a patient ever has a dementia in the course of their study 
DEM_EV = merged.loc[:, ["participant_id", 'dementia']].groupby("participant_id").agg(func=max)

#Join to main HY dataframe 
DEM_merged = merged.join(DEM_EV, on = "participant_id", rsuffix="_EV").reset_index(drop = True)

DEM_merged

,participant_id,visit_name,visit_month,moca_total_score,mms112_total_score,FID,IID,sex,age_at_baseline,age_at_diagnosis,YEARS_DIAGNOSIS_TO_BL,YEARS_FROM_DIAGNOSIS,MOCA21,MMSE26,MOCA_MMSE_cutoffs,dementia,dementia_EV
0,BF-1002,M0,0.0,29.0,<NA>,BF-1002,BF-1002,2,66,61.0,5.0,5.0,0.0,NaN,0.0,0,0
1,BF-1003,M0,0.0,30.0,<NA>,BF-1003,BF-1003,1,61,56.0,5.0,5.0,0.0,NaN,0.0,0,0
2,BF-1004,M0,0.0,28.0,<NA>,BF-1004,BF-1004,1,62,55.0,7.0,7.0,0.0,NaN,0.0,0,0
3,BF-1006,M0,0.0,25.0,<NA>,BF-1006,BF-1006,1,59,52.0,7.0,7.0,0.0,NaN,0.0,0,0
4,BF-1008,M0,0.0,29.0,<NA>,BF-1008,BF-1008,2,69,65.0,4.0,4.0,0.0,NaN,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7455,HB-PD_INVZW684KJU,M0,0.0,NaN,29,HB-PD_INVZW684KJU,HB-PD_INVZW684KJU,1,70,69.0,1.0,1.0,NaN,0.0,0.0,0,0
7456,HB-PD_INVZX194BGG,M0,0.0,NaN,29,HB-PD_INVZX194BGG,HB-PD_INVZX194BGG,1,70,62.0,8.0,8.0,NaN,0.0,0.0,0,0
7457,HB-PD_INVZX600VYV,M0,0.0,NaN,30,HB-PD_INVZX600VYV,HB-PD_INVZX600VYV,2,61,59.0,2.0,2.0,NaN,0.0,0.0,0,0
7458,HB-PD_INVZY881DN4,M0,0.0,NaN,23,HB-PD_INVZY881DN4,HB-PD_INVZY881DN4,1,70,70.0,0.0,0.0,NaN,1.0,1.0,1,1


In [112]:
#Now want to calculate the time to event for dementia

#First filter for individuals who do not ever meet dementia, and take the last visit age where H&Y3+ was not met
DEM_no = (DEM_merged[(DEM_merged.dementia_EV == 0) & (DEM_merged.dementia == 0)].drop_duplicates(subset=["participant_id"],keep="last")
                               .rename({"YEARS_FROM_DIAGNOSIS":"dementia_TIME_FROM_DIAGNOSIS"}, axis=1))

#Secondly filter for individuals who have meet dementia, and take the FIRST visit age where dementia was met
DEM_yes = (DEM_merged[(DEM_merged.dementia_EV == 1) & (DEM_merged.dementia == 1)].drop_duplicates(subset=["participant_id"],keep="first")
                               .rename({"YEARS_FROM_DIAGNOSIS":"dementia_TIME_FROM_DIAGNOSIS"}, axis=1))

#Join dataframes - concatenate (add rows)
DEMENTIA = pd.concat([DEM_no, DEM_yes]).reset_index(drop = True)

#Check for duplicates
DEMENTIA_dup = DEMENTIA[DEMENTIA.duplicated(['participant_id'], keep = False)]
n_dup = len(DEMENTIA_dup)
print(str(n_dup) + ' duplicated')

#If a patient has dementia at baseline (visit_month == 0), they are left censored - code as 2 
DEMENTIA.loc[:,"dementia_EV"] = DEMENTIA.dementia_EV.mask((DEMENTIA.visit_month == 0) & (DEMENTIA.dementia == 1), 2)

0 duplicated


In [113]:
#Now select relevant columns - only one row per participant
DEMENTIA_export = DEMENTIA.copy()
DEMENTIA_export = DEMENTIA_export.loc[:, ['FID', 'IID', 'age_at_diagnosis', 'age_at_baseline', 'sex', 'dementia_EV', 'dementia_TIME_FROM_DIAGNOSIS']]

In [114]:
DEMENTIA_export.dementia_EV.value_counts()

0    2427
2     200
1     182
Name: dementia_EV, dtype: int64

In [115]:
DEMENTIA_export.head()

,FID,IID,age_at_diagnosis,age_at_baseline,sex,dementia_EV,dementia_TIME_FROM_DIAGNOSIS
0,BF-1002,BF-1002,61.0,66,2,0,5.0
1,BF-1003,BF-1003,56.0,61,1,0,5.0
2,BF-1004,BF-1004,55.0,62,1,0,7.0
3,BF-1006,BF-1006,52.0,59,1,0,7.0
4,BF-1008,BF-1008,65.0,69,2,0,4.0


In [116]:
#Write as output
#Note this is all the participants with clinical data, not just our list of PD only participants
with open('/home/jupyter/clinical_data_v2-5/dementia_allparticipants.tab', 'w') as f:
    f.write(DEMENTIA_export.to_csv(index=False, sep='\t'))

Now run survival analysis in R

In [34]:
%%R

dementia_results_list <- list()

#Loop over each pathway and cohort
for (pathway in pathways) {
    for (cohort in cohorts) {
        i <- length(dementia_results_list)
        dementia_results_list[[i+1]] <- analyse_surv(pathway, cohort, "dementia_allparticipants", "dementia")
    }
}

In [35]:
%%R

#Combine lists into a single dataframe
dementia_results <- do.call(rbind, dementia_results_list)

#Set column names
colnames(dementia_results) <- c("pathway", "study", "Coeff", "se", "Pvalue", "Cox.zphPVal", "ov.lik.ratio","logrank", "r2", "N")

#Make into dataframe
dementia_results <- as.data.frame(dementia_results)

#Save output
write.table(dementia_results, "/home/jupyter/pathways/pathways_dementia_results.txt",
           sep = "\t", row.names = F, col.names = T, quote = F)

dementia_results

           pathway   study                Coeff                se
1  adaptive_immune    PPMI   -0.113898994153847 0.110530123954694
2  adaptive_immune    PDBP    0.158662380068633 0.138302942983449
3  adaptive_immune     HBS                 <NA>              <NA>
4  adaptive_immune     LCC                 <NA>              <NA>
5  adaptive_immune  Steady                 <NA>              <NA>
6  adaptive_immune    Sure                 <NA>              <NA>
7  adaptive_immune BioFIND                 <NA>              <NA>
8  alpha_synuclein    PPMI    0.080730518315116 0.124340273367051
9  alpha_synuclein    PDBP -0.00310159809524902 0.144348455474514
10 alpha_synuclein     HBS                 <NA>              <NA>
11 alpha_synuclein     LCC                 <NA>              <NA>
12 alpha_synuclein  Steady                 <NA>              <NA>
13 alpha_synuclein    Sure                 <NA>              <NA>
14 alpha_synuclein BioFIND                 <NA>              <NA>
15   innat

# Analyse pathways vs PDQ8

Query PDQ39 data

In [119]:
pdq_query = f"""

SELECT *
FROM `{BQ_RELEASE_DATASET}.PDQ_39`

"""
pdq = bq_query(pdq_query)

In [120]:
#Select just PDQ8 columns
pdq8 = pdq.copy()
pdq8 = pdq8.loc[:, ['participant_id', 'visit_name', 'visit_month', 'pdq39_07_getting_around_in_public',
                   'pdq39_12_dressing', 'pdq39_17_depressed', 'pdq39_27_close_personal_relations',
                   'pdq39_31_problem_with_concentration', 'pdq39_35_unable_to_communicate', 'pdq39_37_muscle_cramps',
                   'pdq39_25_embarassed_in_public']]

pdq8.head()

,participant_id,visit_name,visit_month,pdq39_07_getting_around_in_public,pdq39_12_dressing,pdq39_17_depressed,pdq39_27_close_personal_relations,pdq39_31_problem_with_concentration,pdq39_35_unable_to_communicate,pdq39_37_muscle_cramps,pdq39_25_embarassed_in_public
0,HB-PD_INVCA430NAZ,M0,0.0,Never,Often,Occasionally,Occasionally,Never,Sometimes,Occasionally,Often
1,PD-PDGR243KPT,M12,12.0,Never,Never,Never,Occasionally,Never,Often,Occasionally,Never
2,PD-PDKG339XM0,M0,0.0,Never,Never,Never,Never,Occasionally,Often,Never,Never
3,PD-PDMZ936CDD,M0,0.0,Never,Never,Occasionally,Often,Never,Always or cannot do at all,Occasionally,Occasionally
4,PD-PDYE025CNY,M36,36.0,Never,Never,Never,Never,Occasionally,Always or cannot do at all,Never,Never


In [121]:
#Recode variables as numeric (0 = Never, 1 = Occasionally, 2 = Sometimes, 3 = Often, 4 = Always)

#Make dictionary to map values (text to numeric)
Dict_PDQ = {'Never':0, 'Occasionally':1, 'Sometimes':2, 'Often':3, 'Always or cannot do at all':4}

pdq8['pdq39_07_getting_around_in_public'] = pdq8.pdq39_07_getting_around_in_public.map(Dict_PDQ).astype(float)
pdq8['pdq39_12_dressing'] = pdq8.pdq39_12_dressing.map(Dict_PDQ).astype(float)
pdq8['pdq39_17_depressed'] = pdq8.pdq39_17_depressed.map(Dict_PDQ).astype(float)
pdq8['pdq39_27_close_personal_relations'] = pdq8.pdq39_27_close_personal_relations.map(Dict_PDQ).astype(float)
pdq8['pdq39_31_problem_with_concentration'] = pdq8.pdq39_31_problem_with_concentration.map(Dict_PDQ).astype(float)
pdq8['pdq39_35_unable_to_communicate'] = pdq8.pdq39_35_unable_to_communicate.map(Dict_PDQ).astype(float)
pdq8['pdq39_37_muscle_cramps'] = pdq8.pdq39_37_muscle_cramps.map(Dict_PDQ).astype(float)
pdq8['pdq39_25_embarassed_in_public'] = pdq8.pdq39_25_embarassed_in_public.map(Dict_PDQ).astype(float)

In [122]:
#Make total PDQ8 column
#If there are any missing values, return NaN
pdq8['PDQ8_TOTAL'] = pdq8.iloc[:, 3:].sum(min_count = 8, axis = 1)

In [123]:
pdq8 = pdq8.sort_values(by = ['participant_id', 'visit_month'])

#Remove duplicated visit entries - rows with #
pdq8_nodupli = pdq8.copy()
pdq8_nodupli = pdq8_nodupli[~pdq8_nodupli.visit_name.str.contains('#')]

#Check for any other duplicated entries by participant id and visit month
pdq8_nodupli[pdq8_nodupli.duplicated(['participant_id', 'visit_month'], keep = False)]

,participant_id,visit_name,visit_month,pdq39_07_getting_around_in_public,pdq39_12_dressing,pdq39_17_depressed,pdq39_27_close_personal_relations,pdq39_31_problem_with_concentration,pdq39_35_unable_to_communicate,pdq39_37_muscle_cramps,pdq39_25_embarassed_in_public,PDQ8_TOTAL
1554,SU-32242,M24,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1555,SU-32242,M24,24.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0
1906,SU-33113,M24,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1907,SU-33113,M24,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1908,SU-33114,M24,24.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1909,SU-33114,M24,24.0,0.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,5.0
1980,SU-33368,M18,18.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1981,SU-33368,M18,18.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [124]:
#There are still a few individuals with duplicated entries by participant_id and visit_month with different MoCA scores
#Just keep the first datapoint
pdq8_nodupli_final = pdq8_nodupli.copy()
pdq8_nodupli_final = pdq8_nodupli_final.drop_duplicates(subset = ['participant_id', 'visit_month'], keep = 'first')

In [125]:
pdq8_nodupli_final.head()

,participant_id,visit_name,visit_month,pdq39_07_getting_around_in_public,pdq39_12_dressing,pdq39_17_depressed,pdq39_27_close_personal_relations,pdq39_31_problem_with_concentration,pdq39_35_unable_to_communicate,pdq39_37_muscle_cramps,pdq39_25_embarassed_in_public,PDQ8_TOTAL
0,HB-PD_INVCA430NAZ,M0,0.0,0.0,3.0,1.0,1.0,0.0,2.0,1.0,3.0,11.0
2328,HB-PD_INVCU917HHH,M0,0.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,24.0
49,HB-PD_INVFP220NZU,M0,0.0,1.0,1.0,3.0,NaN,3.0,1.0,0.0,1.0,NaN
50,HB-PD_INVGH702HGH,M0,0.0,1.0,3.0,1.0,3.0,3.0,1.0,1.0,3.0,16.0
51,HB-PD_INVGT912XMQ,M0,0.0,1.0,1.0,1.0,3.0,3.0,3.0,1.0,1.0,14.0


In [126]:
#Write as output
pdq8_export = pdq8_nodupli_final.copy()
pdq8_export = pdq8_export.loc[:, ['participant_id', 'visit_name', 'visit_month', 'PDQ8_TOTAL']]

with open('/home/jupyter/clinical_data_v2-5/PDQ8_allparticipants.tab', 'w') as f:
    f.write(pdq8_export.to_csv(index=False, sep='\t'))

In [147]:
pdq8_export.head()

,participant_id,visit_name,visit_month,PDQ8_TOTAL
0,HB-PD_INVCA430NAZ,M0,0.0,11.0
2328,HB-PD_INVCU917HHH,M0,0.0,24.0
49,HB-PD_INVFP220NZU,M0,0.0,NaN
50,HB-PD_INVGH702HGH,M0,0.0,16.0
51,HB-PD_INVGT912XMQ,M0,0.0,14.0


Now analyse in mixed model in R

In [13]:
%%R

pdq8_results_list <- list()

#Loop over each pathway and cohort
for (pathway in pathways) {
    for (cohort in cohorts) {
        print(cohort)
        i <- length(pdq8_results_list)
        pdq8_results_list[[i+1]] <- analyse_cont(pathway, cohort, "PDQ8_allparticipants", "PDQ8_TOTAL")

    }
}

[1] "PPMI"
  mean sd
1  NaN NA
[1] "PDBP"
      mean       sd
1 5.747978 5.084956
[1] "HBS"
      mean       sd
1 17.66667 4.830459
[1] "LCC"
  mean sd
1  NaN NA
[1] "Steady"
  mean sd
1  NaN NA
[1] "Sure"
      mean       sd
1 2.149798 2.706104
[1] "BioFIND"
  mean sd
1  NaN NA
[1] "PPMI"
  mean sd
1  NaN NA
[1] "PDBP"
      mean       sd
1 5.747978 5.084956
[1] "HBS"
      mean       sd
1 17.66667 4.830459
[1] "LCC"
  mean sd
1  NaN NA
[1] "Steady"
  mean sd
1  NaN NA
[1] "Sure"
      mean       sd
1 2.149798 2.706104
[1] "BioFIND"
  mean sd
1  NaN NA
[1] "PPMI"
  mean sd
1  NaN NA
[1] "PDBP"
      mean       sd
1 5.747978 5.084956
[1] "HBS"
      mean       sd
1 17.66667 4.830459
[1] "LCC"
  mean sd
1  NaN NA
[1] "Steady"
  mean sd
1  NaN NA
[1] "Sure"
      mean       sd
1 2.149798 2.706104
[1] "BioFIND"
  mean sd
1  NaN NA
[1] "PPMI"
  mean sd
1  NaN NA
[1] "PDBP"
      mean       sd
1 5.747978 5.084956
[1] "HBS"
      mean       sd
1 17.66667 4.830459
[1] "LCC"
  mean sd
1  NaN N

In [37]:
%%R

#Combine lists into a single dataframe
pdq8_results <- do.call(rbind, pdq8_results_list)

#Set column names
colnames(pdq8_results) <- c("pathway", "cohort", "Coeff", "se", "df", "t_value", "Pvalue", "N_obs", "N_inds")

#Make into dataframe
pdq8_results <- as.data.frame(pdq8_results)

#Save output
write.table(pdq8_results, "/home/jupyter/pathways/pathways_pdq8_results.txt",
           sep = "\t", row.names = F, col.names = T, quote = F)

pdq8_results

           pathway  cohort                Coeff                 se
1  adaptive_immune    PPMI                 <NA>               <NA>
2  adaptive_immune    PDBP   0.0635941979669745 0.0322937651133829
3  adaptive_immune     HBS                 <NA>               <NA>
4  adaptive_immune     LCC                 <NA>               <NA>
5  adaptive_immune  Steady                 <NA>               <NA>
6  adaptive_immune    Sure    -0.01818439043883 0.0586729153713862
7  adaptive_immune BioFIND                 <NA>               <NA>
8  alpha_synuclein    PPMI                 <NA>               <NA>
9  alpha_synuclein    PDBP     0.06305876506887 0.0326873437208003
10 alpha_synuclein     HBS                 <NA>               <NA>
11 alpha_synuclein     LCC                 <NA>               <NA>
12 alpha_synuclein  Steady                 <NA>               <NA>
13 alpha_synuclein    Sure  -0.0319240324930557 0.0577936081145065
14 alpha_synuclein BioFIND                 <NA>               

# Analyse pathways vs RBD (never vs. ever)

Query RBD Mayo questionnaire data

In [129]:
rbd_Mayo_query = f"""

SELECT *
FROM `{BQ_RELEASE_DATASET}.REM_Sleep_Behavior_Disorder_Questionnaire_Mayo`

"""
rbd_Mayo = bq_query(rbd_Mayo_query)

In [130]:
rbd_Mayo.head()

,participant_id,GUID,visit_name,visit_month,msq_info_source,msq_interviewee_live_with_subject,msq_interviewee_sleep_same_room,msq_distracting_sleep_behaviors,msq01_act_out_dreams,msq01a_act_out_years,...,msq02_legs_jerk,msq03_restless_legs,msq03a_leg_sensations_decrease,msq03b_time_leg_sensations_worst,msq04b_walked_asleep,msq05_snorted_awake,msq06_stop_breathing,msq06a_treated_for_stop_breathing,msq07_leg_cramps,msq08_rate_of_alertness
0,PD-PDAX486GDU,PDAX486GDU,M0,0.0,Subject served as informant,None,None,None,None,NaN,...,None,None,None,None,None,None,None,None,None,<NA>
1,PD-PDBC294LL7,PDBC294LL7,M0,0.0,Informant,None,None,None,None,NaN,...,None,None,None,None,None,None,None,None,None,<NA>
2,PD-PDBK615WC6,PDBK615WC6,M24,24.0,None,None,None,None,None,NaN,...,None,None,None,None,None,None,None,None,None,<NA>
3,PD-PDCZ531JKD,PDCZ531JKD,M0,0.0,Subject served as informant,None,None,None,None,NaN,...,No,Yes,No,None,Yes,No,No,None,Yes,8
4,PD-PDDW009MK9,PDDW009MK9,M12,12.0,None,None,None,None,None,NaN,...,None,None,None,None,None,None,None,None,None,<NA>


In [131]:
rbd_Mayo.msq01_act_out_dreams.value_counts()

No     2489
Yes    1285
Name: msq01_act_out_dreams, dtype: int64

In [132]:
#Select just relevant columns - only question 1 is used to diagnose RBD
rbd_Mayo = rbd_Mayo.loc[:, ['participant_id', 'visit_name', 'visit_month', 'msq01_act_out_dreams']]

#Make binary variable for RBD
rbd_Mayo['RBD_MAYO'] = [np.nan if pd.isna(i)
                          else 1 if i == "Yes"
                          else 0 for i in rbd_Mayo.msq01_act_out_dreams]

In [133]:
rbd_Mayo.RBD_MAYO.value_counts()

0.0    2489
1.0    1285
Name: RBD_MAYO, dtype: int64

In [134]:
#Remove duplicated visit entries - rows with #
rbd_Mayo_nodupli = rbd_Mayo.copy()
rbd_Mayo_nodupli = rbd_Mayo_nodupli[~rbd_Mayo_nodupli.visit_name.str.contains('#')]

#Check for any other duplicated entries by participant id and visit month
rbd_Mayo_nodupli[rbd_Mayo_nodupli.duplicated(['participant_id', 'visit_month'], keep = False)]

,participant_id,visit_name,visit_month,msq01_act_out_dreams,RBD_MAYO
1987,SU-33113,M24,24.0,No,0.0
2898,SU-33113,M24,24.0,Yes,1.0


In [135]:
#There are still a few individuals with duplicated entries by participant_id and visit_month with different MoCA scores
#Just keep the first datapoint
rbd_Mayo_nodupli_final = rbd_Mayo_nodupli.copy()
rbd_Mayo_nodupli_final = rbd_Mayo_nodupli_final.drop_duplicates(subset = ['participant_id', 'visit_month'], keep = 'first')

In [136]:
rbd_Mayo_nodupli_final.head()

,participant_id,visit_name,visit_month,msq01_act_out_dreams,RBD_MAYO
0,PD-PDAX486GDU,M0,0.0,None,NaN
1,PD-PDBC294LL7,M0,0.0,None,NaN
2,PD-PDBK615WC6,M24,24.0,None,NaN
3,PD-PDCZ531JKD,M0,0.0,None,NaN
4,PD-PDDW009MK9,M12,12.0,None,NaN


Query RBDSQ (Stiasny-Kolster) data

In [137]:
RBDSQ_query = f"""

SELECT
participant_id, visit_name, visit_month, rbd_summary_score
FROM `{BQ_RELEASE_DATASET}.REM_Sleep_Behavior_Disorder_Questionnaire_Stiasny_Kolster`

"""
RBDSQ = bq_query(RBDSQ_query)

In [138]:
#Make new variable if summary score is >=6
RBDSQ['RBD_RBDSQ'] = [np.nan if pd.isna(i)
                          else 1 if i >=6
                          else 0 for i in RBDSQ.rbd_summary_score]

#Remove duplicated visit entries - rows with #
RBDSQ_nodupli = RBDSQ.copy()
RBDSQ_nodupli = RBDSQ_nodupli[~RBDSQ_nodupli.visit_name.str.contains('#')]

#Check for any other duplicated entries by participant id and visit month
RBDSQ_nodupli[RBDSQ_nodupli.duplicated(['participant_id', 'visit_month'], keep = False)]

,participant_id,visit_name,visit_month,rbd_summary_score,RBD_RBDSQ


In [139]:
RBDSQ.RBD_RBDSQ.value_counts()

0.0    7683
1.0    2527
Name: RBD_RBDSQ, dtype: int64

In [140]:
RBDSQ_nodupli.head()

,participant_id,visit_name,visit_month,rbd_summary_score,RBD_RBDSQ
0,HB-PD_INVAB465GYE,M0,0.0,0,0.0
1,HB-PD_INVAG058KWW,M0,0.0,0,0.0
2,HB-PD_INVAK106DV5,M0,0.0,0,0.0
3,HB-PD_INVAR135FG7,M0,0.0,0,0.0
4,HB-PD_INVBC055WLN,M0,0.0,0,0.0


In [141]:
#Merge RBD datasets
merged = RBDSQ_nodupli.merge(rbd_Mayo_nodupli_final, on = (['participant_id', 'visit_name', 'visit_month']), how = 'outer')

#Check whether there are any individuals with multiple entires for visit_name and visit_month
#Because we merged on participant_id + visit_name + visit_month, there may be some individuals with the same visit name but different visit months
merged[merged.duplicated(['participant_id', 'visit_name'])]

,participant_id,visit_name,visit_month,rbd_summary_score,RBD_RBDSQ,msq01_act_out_dreams,RBD_MAYO


In [142]:
#Check individuals who have data for both RBD questionnaires
both = RBDSQ_nodupli.merge(rbd_Mayo_nodupli, on = (['participant_id', 'visit_name', 'visit_month']), how = 'inner')
both.head()

,participant_id,visit_name,visit_month,rbd_summary_score,RBD_RBDSQ,msq01_act_out_dreams,RBD_MAYO
0,PD-PDBY238FWE,M0,0.0,0,0.0,No,0.0
1,PD-PDED633MRU,M0,0.0,0,0.0,No,0.0
2,PD-PDEH654NL2,M0,0.0,0,0.0,No,0.0
3,PD-PDFW311WR6,M0,0.0,0,0.0,No,0.0
4,PD-PDNF237HNT,M0,0.0,0,0.0,No,0.0


In [143]:
#Make a variable coding for RBD if EITHER the RBDSQ or Mayo criteria is met
merged['RBD_sum'] = merged.fillna(0)['RBD_RBDSQ'] + merged.fillna(0)['RBD_MAYO']

#Now make a combined variable which codes dementia if EITHER MOCA or MMSE are below cutoffs
merged['RBD_final'] = [np.nan if pd.isna(i)
                else 1 if i >= 1
                else 0 for i in merged.RBD_sum]

#However if both RBDSQ and MAYO criteria are NA, replace RBD_final as NA
merged.loc[:, 'RBD_final'] = merged.RBD_final.mask((merged.RBD_RBDSQ.isnull()) & (merged.RBD_MAYO.isnull()), np.nan)

In [144]:
#Make a variable for RBD EVER because we will analyse RBD as never vs. ever in a logistic regression
RBD_export = merged.copy()
RBD_export = RBD_export.loc[:, ['participant_id', 'RBD_final']].groupby('participant_id').agg(func = max)
RBD_export = RBD_export.reset_index()

In [145]:
#Write as output
with open('/home/jupyter/clinical_data_v2-5/RBD_allparticipants.tab', 'w') as f:
    f.write(RBD_export.to_csv(index=False, sep='\t'))

In [146]:
RBD_export.head()

,participant_id,RBD_final
0,BF-1001,0.0
1,BF-1002,0.0
2,BF-1003,0.0
3,BF-1004,0.0
4,BF-1005,0.0


Analyse in R with logistic regression

In [14]:
%%R

#Make list of pathways
pathways <- c("adaptive_immune", "alpha_synuclein", "innate_immune", "lysosome", "endocytosis",
              "microglia", "monocytes", "mitochondria")

#Make list of cohorts - excluding LBD as these samples have been excluded from the genetic dataset
cohorts <- c("PPMI", "PDBP", "HBS", "LCC", "Steady", "Sure", "BioFIND")

#Make function to analyse longitudinal outcomes using mixed effects models
#df is the name of the .tab file in /home/jupyter/clinical_data_v2-5/ which has the data in long format
#df = "mdsupdrs3_allparticipants"
#outcome is the variable name of the outcome within df
#outcome = "mds_updrs_part_ii_summary_score"

analyse_binomial <- function(pathway, cohort, df, outcome) {
    
        #Read in data for PRS, age at diagnosis, cohort, and genetic PCs
        PRS <- read.table(paste("/home/jupyter/pathways/", cohort, ".", pathway, "_PRS_final.txt", sep = ""), 
                          sep = "\t", header = T)
    
        data <- read.table(paste("/home/jupyter/clinical_data_v2-5/", df, ".tab", sep = ""), 
                             sep = "\t", header = T)
    
        aao <- read.table("/home/jupyter/clinical_data_v2-5/PD_cases_agedx.tab", sep = "\t", header = T)

        demo <- read.table("/home/jupyter/clinical_data_v2-5/PD_cases_start_end_dx_sex.tab", sep = "\t", header = T)

        #Read in genetic PCs
        pcs <- read.table(paste("/home/jupyter/genetic_data_v2-5/", cohort, ".pca.eigenvec", sep = ""))
        colnames(pcs) <- c("FID", "IID", paste("PC", 1:10, sep = ""))

        #Merge - in long format, starting from the longitudinal outcome data
        merged <- data %>%
        inner_join(PRS, by = "participant_id") %>%
        inner_join(aao, by = c("participant_id" = "FID")) %>%
        inner_join(demo, by = "IID") %>%
        inner_join(pcs, by = c("FID", "IID"))

        #Select the outcome variable from the function argument
        merged$y <- merged[[outcome]]
    
        #Select just relevant columns
        #This cannot be done with dplyr::select
        merged_selected <- merged[, c('y', 'participant_id', 'study', 'PRS_z', 'sex', 
                                      'age_at_baseline', 'age_at_diagnosis',
                                      'PC1', 'PC2', 'PC3', 'PC4', 'PC5')]
    
        
        #Remove individuals missing any rows
        merged_selected_nomiss <- merged_selected %>%
            drop_na()
    
        #Count how many individuals had RBD never vs. ever
        print(merged_selected_nomiss %>%
            group_by(y) %>%
            summarise(count = n()))
    
        N_inds <- dim(merged_selected_nomiss)[1]

        #For mixed effects models, we usually need 2 x number of individuals because we are estimating random effects for both the intercept and slope
        #If the number of observations is more than 2 times the number of individuals (this is because we have N_inds * 2 random effects, one for the intercept and one for the slope)
        if (N_inds > 0) {
            
            #Run GLM
            glm_RBD_PRS <- glm(y ~ PRS_z + sex + age_at_diagnosis
                       + PC1 + PC2 + PC3 + PC4 + PC5,
                      data = merged_selected_nomiss,
                      family = binomial())

            
            #Save model coefficients to results table
            summary <-  c(pathway,
                          cohort,
                          summary(glm_RBD_PRS)$coefficients[2,1],
                          summary(glm_RBD_PRS)$coefficients[2,2],
                          summary(glm_RBD_PRS)$coefficients[2,4],
                          nobs(glm_RBD_PRS))
            
            } else {
            
        
            #If the dataframe is empty, print NA for summary stat
            summary <- c(pathway,
                        cohort,
                        rep(NA, 3),
                        N_inds)
            
            }
        
                     
    return(summary)
        
}

In [15]:
%%R

rbd_results_list <- list()

#Loop over each pathway and cohort
for (pathway in pathways) {
    for (cohort in cohorts) {
        print(cohort)
        i <- length(rbd_results_list)
        rbd_results_list[[i+1]] <- analyse_binomial(pathway, cohort, "RBD_allparticipants", "RBD_final")

    }
}

[1] "PPMI"
# A tibble: 2 × 2
      y count
  <dbl> <int>
1     0   158
2     1   271
[1] "PDBP"
# A tibble: 2 × 2
      y count
  <dbl> <int>
1     0   317
2     1   422
[1] "HBS"
# A tibble: 2 × 2
      y count
  <dbl> <int>
1     0   256
2     1   106
[1] "LCC"
# A tibble: 2 × 2
      y count
  <dbl> <int>
1     0    36
2     1    23
[1] "Steady"
# A tibble: 0 × 2
# … with 2 variables: y <dbl>, count <int>
[1] "Sure"
# A tibble: 2 × 2
      y count
  <dbl> <int>
1     0   149
2     1    93
[1] "BioFIND"
# A tibble: 2 × 2
      y count
  <dbl> <int>
1     0    52
2     1    40
[1] "PPMI"
# A tibble: 2 × 2
      y count
  <dbl> <int>
1     0   158
2     1   271
[1] "PDBP"
# A tibble: 2 × 2
      y count
  <dbl> <int>
1     0   317
2     1   422
[1] "HBS"
# A tibble: 2 × 2
      y count
  <dbl> <int>
1     0   256
2     1   106
[1] "LCC"
# A tibble: 2 × 2
      y count
  <dbl> <int>
1     0    36
2     1    23
[1] "Steady"
# A tibble: 0 × 2
# … with 2 variables: y <dbl>, count <int>
[1]

In [40]:
%%R

#Combine lists into a single dataframe
rbd_results <- do.call(rbind, rbd_results_list)

#Set column names
colnames(rbd_results) <- c("pathway", "cohort", "Coeff", "se", "Pvalue", "N_inds")

#Make into dataframe
rbd_results <- as.data.frame(rbd_results)

#Save output
write.table(rbd_results, "/home/jupyter/pathways/pathways_rbd_results.txt",
           sep = "\t", row.names = F, col.names = T, quote = F)

rbd_results

           pathway  cohort                Coeff                 se
1  adaptive_immune    PPMI  -0.0338571202839722  0.100979876312308
2  adaptive_immune    PDBP    0.126202103234058 0.0758834375739297
3  adaptive_immune     HBS    0.167901846139867  0.111436204786255
4  adaptive_immune     LCC    0.333136332895856  0.300032347100603
5  adaptive_immune  Steady                 <NA>               <NA>
6  adaptive_immune    Sure -0.00624871776123015  0.145126304153982
7  adaptive_immune BioFIND   -0.364072832263849  0.255099442421872
8  alpha_synuclein    PPMI  -0.0857466846625837  0.103728735724328
9  alpha_synuclein    PDBP  -0.0811883659301612 0.0759284161142755
10 alpha_synuclein     HBS   0.0951744035004351  0.117969246595435
11 alpha_synuclein     LCC    0.499183861767909   0.32806139337819
12 alpha_synuclein  Steady                 <NA>               <NA>
13 alpha_synuclein    Sure   -0.168850245384872  0.141660143752035
14 alpha_synuclein BioFIND   -0.340629144818277  0.24639566726

In [41]:
%%bash
ls /home/jupyter/pathways/

adaptive_immune.bed
adaptive_immune.bim
adaptive_immune_chrbp.txt
adaptive_immune_extract_hg38.txt
adaptive_immune.fam
adaptive_immune.log
adaptive_immune_PD_PRS_Chang23andMe.all_score
adaptive_immune_PD_PRS_Chang23andMe.log
adaptive_immune_PD_PRS_Chang23andMe.mismatch
adaptive_immune_PD_PRS_Chang23andMe.prsice
adaptive_immune.updated_names.bed
adaptive_immune.updated_names.bim
adaptive_immune.updated_names.fam
adaptive_immune.updated_names.log
adaptive_immune.updated_names.no_dupli.bed
adaptive_immune.updated_names.no_dupli.bim
adaptive_immune.updated_names.no_dupli.fam
adaptive_immune.updated_names.no_dupli.log
alpha_synuclein.bed
alpha_synuclein.bim
alpha_synuclein_chrbp.txt
alpha_synuclein_extract_hg38.txt
alpha_synuclein.fam
alpha_synuclein.log
alpha_synuclein_PD_PRS_Chang23andMe.all_score
alpha_synuclein_PD_PRS_Chang23andMe.log
alpha_synuclein_PD_PRS_Chang23andMe.prsice
alpha_synuclein.updated_names.bed
alpha_synuclein.updated_names.bim
alpha_synuclein.updated_names.fam
alpha_syn

# Copy results files to workspace bucket

In [42]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} cp -r /home/jupyter/pathways/pathways_* {WORKSPACE_BUCKET}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} cp -r /home/jupyter/pathways/n_snps_in_pathwayPRS_summary.tab {WORKSPACE_BUCKET}')


Executing: gsutil -u manuela-norway cp -r /home/jupyter/pathways/pathways_* gs://fc-2f037edd-3b8a-43cc-8283-b6704707b181
Copying file:///home/jupyter/pathways/pathways_agediagnosis_results.txt [Content-Type=text/plain]...
Copying file:///home/jupyter/pathways/pathways_dementia_results.txt [Content-Type=text/plain]...
Copying file:///home/jupyter/pathways/pathways_HY3_results.txt [Content-Type=text/plain]...
Copying file:///home/jupyter/pathways/pathways_mdsupdrs2_results.txt [Content-Type=text/plain]...
- [4 files][ 15.8 KiB/ 15.8 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file:///home/jupyter/pathways/pathways_mdsupdrs3_results.txt [Content-Type=text/plain]...
Copying file:///home/jupyter/pathways/pathways_moca_results.